# Conceptual Analysis Workbook for 6/29
### Plan:
* Load all three models from text + define new embed getters
* Update helper methods to include all three models
* Conduct angle tests w/ GPT, OPT, and T5
* Define + discuss complexity tests
* Conduct complexity tests w/ OPT and T5

In [13]:
import os
import torch
import openai
import numpy as np

import sentencepiece    # necessary for proper t5 init.
from transformers import T5Tokenizer, T5EncoderModel, GPT2Tokenizer, OPTModel

from sklearn.metrics.pairwise import cosine_similarity

# api key set in conda env.
openai.api_key = os.getenv('OPENAI_API_KEY')

## Load Models

### Vocab

In [14]:
vocab = []
with open('./vocab/expanded_vocab.txt', 'r') as f:
    for line in f:
        vocab.append(line.strip())

len(vocab)

5124

### GPT-3

In [15]:
# Loading saved embeddings from GPT-3 Ada
# Ada embeds: 1024 dims
# Babbage load included below as comment.
"""
# Babbage embeds: 2048 dims
bab_embeds = []
with open(u'/gpfs/fs1/home/mbarlow6/Desktop/Conceptual-Analysis/barlow/gpt/gpt_babbage.txt', 'r') as f:
    for line in f:
        bab_embeds.append([float(x) for x in line.strip().split()])

model_bab = dict(zip(vocab, bab_embeds))
"""
ada_embeds = []
with open(u'./gpt/gpt_ada.txt', 'r') as f:
    for line in f:
        ada_embeds.append([float(x) for x in line.strip().split()])

model_gpt = dict(zip(vocab, ada_embeds))

In [16]:
# for getting new embeddings from OpenAI
def gpt_embed(text, engine='text-similarity-ada-001'):
    text = text.replace('\n', ' ')
    return openai.Embedding.create(input=[text], engine=engine)['data'][0]['embedding']

### OPT

In [ ]:
# Loding OPT with HuggingFace Transformers. Paramenter size can be changed- ['125m', '350m', '1.3b', '2.7b', '6.7b', '13b', '30b']
# NOTE: 1.3b uses 2.6G of .cache space.

# NOTE: The documentation recommends using the GPT2Tokenizer here.
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('facebook/opt-1.3b', cache_dir='/scratch/mbarlow6/cache')
model_opt_raw = OPTModel.from_pretrained('facebook/opt-1.3b', cache_dir='/scratch/mbarlow6/cache')

In [18]:
opt_embeds = []
with open(u'./opt/1_3B.txt', 'r') as f:
    for line in f:
        opt_embeds.append([float(x) for x in line.strip().split()])
model_opt = dict(zip(vocab, opt_embeds))

In [19]:
# This is how I'm getting the embedding from OPT.
def opt_embed(word, tokenizer=gpt2_tokenizer, model=model_opt_raw, debug=False):
    # prepare inputs and model
    inputs = tokenizer(word, return_tensors="pt") # return pytorch tensors
    if debug:
        print('Tokens Requested:')
        print(tokenizer.batch_decode(inputs.input_ids[0]))
    with torch.no_grad():
        outputs = model(**inputs)

    # get embedding from final layer
    last_hidden_states = outputs.last_hidden_state # I have confirmed this is the same
                                                   # As the strategy from Tianyi

    # remove unnecessary dimension
    embeddings = torch.squeeze(last_hidden_states, dim=0)

    # add all of the vectors together
    final_embed = np.array(embeddings[1])
    if embeddings.shape[0] > 2:
        for next_emb in embeddings[2:]:
            final_embed += np.array(next_emb)
        final_embed /= embeddings.shape[0] - 1
    return final_embed
    # same outcome as strategy in negatives.ipynb

### T5
Encoder-only mean pooling embeddings most likely replicate the behavior we want - see negatives.ipynb loading section for full comment.

In [ ]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-large', cache_dir='/scratch/mbarlow6/cache')
model_t5_raw = T5EncoderModel.from_pretrained('t5-large', cache_dir='/scratch/mbarlow6/cache')

In [21]:
t5_embeds = []
with open('./t5/t5large.txt', 'r') as f:
    for line in f:
        t5_embeds.append([float(x) for x in line.strip().split()])
model_t5 = dict(zip(vocab, t5_embeds))

In [22]:
def t5_embed(text, tokenizer=t5_tokenizer, model=model_t5_raw, debug=False):
    inputs = tokenizer(text, return_tensors='pt')
    if debug:
        print('Tokens Requested:')
        print(tokenizer.batch_decode(inputs.input_ids[0]))
    with torch.no_grad():
        outputs = model(**inputs)
    # get final hidden state, remove extra dim
    embeddings = torch.squeeze(outputs.last_hidden_state, dim=0)
    # average the embedding of each token by dimension, return.
    return np.array(torch.mean(embeddings, dim=0))

In [23]:
# from tqdm.notebook import tqdm
# for i in tqdm(range(len(vocab))):
#     word = vocab[i]
#     em = t5_embed(word)
#     with open('./t5/t5large.txt', 'a') as f:
#         for p in em:
#             f.write(str(p) + ' ')
#         f.write('\n')

## Define Helpers (for angle)

In [24]:
def positive(words, model='gpt'):
    """
    Args:
        words: iterable
        model: 'gpt', 'opt', or 't5'
    Returns:
        Positive (summed vectors) of word embeddings of a given list of words from the specified model. Defaults to GPT-3.
    """
    if isinstance(words, str):
        print(f"You requested the positive of the string \"{words}\". Did you mean [\"{words}\"]?")

    out = 0
    for token in words:
        # convert token to string
        word = str(token)
        flag = word in vocab
        # do model check - least intensive operation to repeat
        if model.lower() == 'gpt':
            # look for token in cached GPT embeds
            if flag:
                ex = model_gpt[word]  # ex for "extracted"
            # if not found, query API
            else:
                ex = gpt_embed(word)
                model_gpt[word] = ex
        elif model.lower() == 'opt':
            if flag:
                ex = model_opt[word]
            else:
                ex = opt_embed(word)
                model_opt[word] = ex
        elif model.lower() == 't5':
            if flag:
                ex = model_t5[word]
            else:
                ex = t5_embed(word)
                model_t5[word] = ex
        else:
            raise ValueError('Please provide either gpt, opt, or t5 as a model choice.')

        # construct positive
        if isinstance(out, int):
            out = np.array(ex).reshape(1, -1)
        else:
            out += np.array(ex).reshape(1, -1)
            
    return out if not isinstance(out, int) else np.array([])

In [25]:
def most_similar(target, phrase=False, topn=10, model='gpt'):
    """
    Args:
        target: str or list[float] (if phrase)
        phrase: bool    -> True if target is vector. Default False.
        topn: int       -> Number of results to keep [0:2+topn]
        model: str      -> 'gpt', 'opt', or 't5'
    Returns:
        Top n most similar words in common vocab to the target vector in the specified model.
    """
    # holder for all similarity scores
    sims = {}

    # initialize starting point
    a = target

    # if no phrase, must convert to embed.
    if not phrase:
        a = positive([a], model)

    # now, we check every B in our vocab
    for term in vocab:
        b = positive([term], model)
        if isinstance(target, str):
            if term != target:
                sims[term] = cosine_similarity(a, b)[0][0]
        elif b.any():
            sims[term] = cosine_similarity(a, b)[0][0]
    
    return list({k: v for k, v in sorted(sims.items(), key=lambda x: x[1], reverse=True)}.items())[:2+topn]

In [26]:
def definition(words, topn=10, model='gpt'):
    """
    Args:
        words: iterable -> Items to be combined into a positive and defined
        topn: int       -> number of words to keep in definiton
        model: str      -> 'gpt', 'opt', or 't5'
    Returns:
        Top N most similar words in the specified model to a positive of the string representations
        of any provided interable, if it's members can be cast to strings. 
    """
    # get phrase
    if isinstance(words, str):
        words = [words]
    phrase = positive(words, model)

    # get definition
    def_of_positive = most_similar(phrase, True, topn, model)

    return def_of_positive

In [27]:
def define(words, topn=10):
    """
    Args:
        words: string or iterable -> Items to be combined into a positive and defined
        topn: int                 -> number of words to keep in definiton
    Returns:
        None. Formats and prints definition from both models.
    """
    print(*words, sep=', ')
    print('GPT-3:')
    print(definition(words, topn, 'gpt'))
    print('------------------------------------------------------')
    print('OPT:')
    print(definition(words, topn, 'opt'))
    print('------------------------------------------------------')
    print('T5:')
    print(definition(words, topn, 't5'))
    print('------------------------------------------------------')


In [28]:
def calculate_similarity(words, target, vec=False, model='gpt'):
    """
    Args:
        words: iterable or vector   -> Items to be combined into a positive and compared,
                                       or already constructed vector of matching dimensions.
        target: str     -> single term to calulate similarity to.
        vec: bool       -> true if 'words' is a vector
        model: str      -> 'gpt', 'opt', or 't5'
    Returns:
        The cosine similarity of the words vector and target term in specified model.
    """
    # get phrase
    phrase = words
    if not vec:
        if isinstance(words, str):
            phrase = positive([words], model)
        else:
            phrase = positive(words, model)
    
    # get target
    target = positive([target], model)

    return cosine_similarity(phrase, target)[0][0]


In [29]:
def sim_test(words, target):
    """
    Args:
        words: iterable, str, or vector -> Items to be combined into a positive and compared,
                                            or already constructed vector of matching dimensions.
        target: str                     -> single term to calulate similarity to.
    Returns:
        None. Formats and prints similarity from both models.
    """
    print(*words, sep=', ', end='')
    print(f' -> {target}')
    print('GPT-3:')
    print(calculate_similarity(words, target, model='gpt'))
    print('------------------------------------------------------')
    print('OPT:')
    print(calculate_similarity(words, target, model='opt'))
    print('------------------------------------------------------')
    print('T5:')
    print(calculate_similarity(words, target, model='t5'))
    print('------------------------------------------------------')

I'm choosing to leave out a function where both A and B are vectors, as that is mostly helpful for phrase vectors we're not constucting ourselves - so I'll include it later if needed.

## Conduct Angle tests

### A - Bachelors, Husbands, and Wives
Words describing spousal relations.

In [30]:
define(['man', 'unmarried'])

man, unmarried
GPT-3:
[('man', 0.9574393871008093), ('unmarried', 0.9574393842477984), ('person', 0.9282141375654523), ('human', 0.9260773583481672), ('married', 0.925716217524681), ('husband', 0.9238631624971028), ('manifest', 0.923207222251275), ('male', 0.9227498746941889), ('couple', 0.9194728139414444), ('what', 0.9178848665087231), ('single', 0.91564771604982), ('one', 0.915483735529741)]
------------------------------------------------------
OPT:
[('man', 0.885996842881478), ('manner', 0.8487586505974158), ('manifest', 0.8305222722582827), ('manage', 0.8198653589266234), ('manuscript', 0.8179447952184589), ('manipulation', 0.7761658724673628), ('manipulate', 0.7733485935553648), ('unmarried', 0.7626662919261634), ('unfortunately', 0.7327823689854197), ('unemployed', 0.731421729052413), ('boyfriend', 0.7143500324551498), ('unlikely', 0.7067586208467709)]
------------------------------------------------------
T5:
[('man', 0.8413566970505227), ('unmarried', 0.8314060001451469), ('h

In [31]:
define(['man', 'married'])

man, married
GPT-3:
[('man', 0.9614419224507509), ('married', 0.9614419219173929), ('husband', 0.9484450418023361), ('marry', 0.9404309015085552), ('marriage', 0.9382387494957314), ('couple', 0.934796971530077), ('spouse', 0.9318137079053036), ('person', 0.9290811125868605), ('human', 0.9233877537015611), ('you', 0.922936509299525), ('manifest', 0.9213617356108426), ('him', 0.9211363475663665)]
------------------------------------------------------
OPT:
[('man', 0.8663384449606284), ('manner', 0.8316395110979301), ('manifest', 0.8252928694287786), ('manuscript', 0.8148737054854491), ('manage', 0.8141060592979465), ('married', 0.7982615376954943), ('manipulate', 0.7798194649556903), ('manipulation', 0.7614170008717042), ('boyfriend', 0.7450165191295177), ('mate', 0.718733994661042), ('meantime', 0.7012631976096041), ('yesterday', 0.6999955894352266)]
------------------------------------------------------
T5:
[('man', 0.8969295782758443), ('married', 0.8912107968059642), ('marry', 0.8086

In [32]:
sim_test(['man', 'unmarried'], 'bachelor')

man, unmarried -> bachelor
GPT-3:
0.903310712489814
------------------------------------------------------
OPT:
0.416106034230743
------------------------------------------------------
T5:
0.6297230350407689
------------------------------------------------------


In [33]:
sim_test(['man', 'married'], 'husband')

man, married -> husband
GPT-3:
0.9484450418023361
------------------------------------------------------
OPT:
0.6647823970515117
------------------------------------------------------
T5:
0.7833614103896772
------------------------------------------------------


In [34]:
define(['woman', 'married'])

woman, married
GPT-3:
[('married', 0.964631336834191), ('woman', 0.9646313364435788), ('wife', 0.9568923973033583), ('spouse', 0.9433161765505673), ('marriage', 0.9425896983373434), ('female', 0.9396147160196819), ('marry', 0.9388715585659668), ('husband', 0.936670166774265), ('couple', 0.936052402761764), ('she', 0.9326070856151396), ('wives', 0.929861649926939), ('lady', 0.9248636675201433)]
------------------------------------------------------
OPT:
[('woman', 0.8858763608258227), ('married', 0.8238625714337076), ('daughter', 0.7797461991340747), ('boyfriend', 0.7657742136832786), ('girlfriend', 0.747518990301832), ('girl', 0.7370161736159565), ('wife', 0.7348894394239303), ('husband', 0.7330106170944921), ('manuscript', 0.7066314937987579), ('grandmother', 0.6871470186954577), ('granddaughter', 0.6830161321535737), ('youngster', 0.6815322787998708)]
------------------------------------------------------
T5:
[('woman', 0.9075679596067301), ('married', 0.9050173807842091), ('marry', 

In [35]:
sim_test(['woman', 'married'], 'wife')

woman, married -> wife
GPT-3:
0.9568923973033583
------------------------------------------------------
OPT:
0.7348894394239303
------------------------------------------------------
T5:
0.7078476354479186
------------------------------------------------------


In [36]:
define(['woman', 'spouse'])

woman, spouse
GPT-3:
[('woman', 0.9693675955226067), ('spouse', 0.9693675949195859), ('wife', 0.9583028946210856), ('female', 0.9382857953373832), ('spouses', 0.9360382550097874), ('husband', 0.9339551354635074), ('couple', 0.9332387684111279), ('she', 0.9309995678243246), ('married', 0.9292578268401397), ('person', 0.9289279426727296), ('wives', 0.9280040650514648), ('lady', 0.9267049168586852)]
------------------------------------------------------
OPT:
[('woman', 0.9122844414270574), ('spouse', 0.7890891090037166), ('boyfriend', 0.7755293045644811), ('spouses', 0.7722972938983756), ('daughter', 0.7696403298943281), ('husband', 0.7688547074091923), ('wife', 0.7541077180663601), ('girlfriend', 0.7450312881001115), ('girl', 0.7420200925794256), ('manipulation', 0.7103418845215251), ('spokeswoman', 0.7095966258520794), ('spokesperson', 0.7075331434493506)]
------------------------------------------------------
T5:
[('spouse', 0.8869849958872391), ('woman', 0.8795312624215301), ('husband

In [37]:
define(['women', 'spouses'])

women, spouses
GPT-3:
[('wives', 0.9678768625462946), ('spouses', 0.9670318160416622), ('spouse', 0.9532600180422997), ('wife', 0.9466426543882364), ('couple', 0.9365391692912451), ('husbands', 0.9345357850323417), ('marriage', 0.9336396632195183), ('married', 0.9307641795242254), ('husband', 0.9303849939766571), ('woman', 0.9292538039789786), ('female', 0.9260824595388291), ('family', 0.922073326447916)]
------------------------------------------------------
OPT:
[('spouses', 0.8378829501747723), ('children', 0.7960743236439928), ('girls', 0.7891043930421603), ('wives', 0.7843908691965316), ('husbands', 0.7337424823483121), ('woman', 0.7321836867679454), ('boys', 0.73120158590451), ('people', 0.7132028237948403), ('spouse', 0.702309046308607), ('daughter', 0.6974442735139585), ('sheep', 0.6903888936584397), ('himself', 0.6838380788687537)]
------------------------------------------------------
T5:
[('spouses', 0.797555026956622), ('girls', 0.6487775654259706), ('woman', 0.633240728564

In [38]:
sim_test(['woman', 'spouse'], 'wife')

woman, spouse -> wife
GPT-3:
0.9583028946210856
------------------------------------------------------
OPT:
0.7541077180663601
------------------------------------------------------
T5:
0.768315428322647
------------------------------------------------------


In [39]:
sim_test(['women', 'spouses'], 'wives')

women, spouses -> wives
GPT-3:
0.9678768625462946
------------------------------------------------------
OPT:
0.7843908691965316
------------------------------------------------------
T5:
0.5515538879521509
------------------------------------------------------


In [40]:
define(['man', 'spouse'])

man, spouse
GPT-3:
[('man', 0.962084172364732), ('spouse', 0.9620841706718597), ('husband', 0.9496874597051836), ('person', 0.9409165214048233), ('couple', 0.9359461095366195), ('spouses', 0.9350059048616294), ('married', 0.9299075956420162), ('human', 0.9283761067489947), ('wife', 0.9260494730406059), ('him', 0.9234024671581352), ('sum', 0.9232893187721536), ('father', 0.922784008432849)]
------------------------------------------------------
OPT:
[('man', 0.8994232336052005), ('manifest', 0.8631178062918794), ('manner', 0.8577256424381958), ('manipulation', 0.8229738037612163), ('manuscript', 0.8183365195446253), ('manage', 0.816225735024274), ('manipulate', 0.7883471711448886), ('spouse', 0.7624252520480261), ('boyfriend', 0.7482258282753127), ('spouses', 0.7385119915625636), ('mate', 0.7081036290860419), ('spokesman', 0.7070451879937989)]
------------------------------------------------------
T5:
[('spouse', 0.8670936489734951), ('man', 0.8620094423423793), ('husband', 0.8448111636

In [41]:
define(['men', 'spouses'])

men, spouses
GPT-3:
[('spouses', 0.9632354267890622), ('husbands', 0.9502174456654946), ('spouse', 0.9408705691171747), ('husband', 0.9379355106158141), ('couple', 0.9344770546948105), ('wives', 0.9268565274462921), ('married', 0.9248361318444289), ('marriage', 0.9241547935439827), ('family', 0.9200518284058132), ('people', 0.9182769860781621), ('they', 0.9153267610273681), ('father', 0.9150406784851493)]
------------------------------------------------------
OPT:
[('spouses', 0.8466080582742529), ('spouse', 0.7456312086417818), ('husbands', 0.7359921769477056), ('boys', 0.7318386411986559), ('wives', 0.7100812747422823), ('girls', 0.7089302302097362), ('teens', 0.6869850909792079), ('manipulation', 0.6827339202935186), ('children', 0.6817313736404076), ('spokesman', 0.6797241741504711), ('spokesperson', 0.678176076395433), ('siblings', 0.6730379879023307)]
------------------------------------------------------
T5:
[('spouses', 0.7703073652693275), ('husbands', 0.7571537894770407), ('w

In [42]:
define(['male', 'spouse'])

male, spouse
GPT-3:
[('male', 0.9623219204673592), ('spouse', 0.9623219190362389), ('husband', 0.9475313396332239), ('couple', 0.9394179702979454), ('spouses', 0.9339890746748664), ('married', 0.9330896147854479), ('person', 0.9280096291952866), ('gender', 0.9267599042731943), ('husbands', 0.9237886381161453), ('female', 0.9237204541199127), ('partner', 0.9236599895812495), ('human', 0.9218679288082892)]
------------------------------------------------------
OPT:
[('male', 0.8732633833191408), ('female', 0.8001403537352472), ('spouse', 0.7735667864111877), ('spouses', 0.7372964706319469), ('boyfriend', 0.6764440225656366), ('spokesperson', 0.6690374961475587), ('spokesman', 0.6614587010296803), ('wife', 0.6584279653989359), ('husband', 0.6539461379697912), ('spokeswoman', 0.6538185643389719), ('daughter', 0.6533869213532835), ('woman', 0.6446877385911055)]
------------------------------------------------------
T5:
[('spouse', 0.8762307891884773), ('husband', 0.7028997578998908), ('wife

In [43]:
define(['male', 'spouses'])

male, spouses
GPT-3:
[('male', 0.9558336643759909), ('spouses', 0.955833657892458), ('spouse', 0.9533496092933855), ('husband', 0.9481375445396195), ('husbands', 0.9427816858036653), ('couple', 0.9390864083595996), ('married', 0.9333003909696297), ('marriage', 0.9278369231760236), ('father', 0.9229110567683492), ('gender', 0.9211675776031061), ('human', 0.9189393222001251), ('house', 0.9172186945240626)]
------------------------------------------------------
OPT:
[('male', 0.8633301696589903), ('spouses', 0.79540061618441), ('female', 0.7933920742821607), ('spouse', 0.7383700694602912), ('husbands', 0.685926507865421), ('boyfriend', 0.6843965207987109), ('spokesperson', 0.6785417931869093), ('daughter', 0.672519954291948), ('wives', 0.670862085449726), ('spokesman', 0.6696620449964078), ('spokeswoman', 0.6633012696520003), ('woman', 0.6586169214626504)]
------------------------------------------------------
T5:
[('male', 0.7867614497901925), ('spouses', 0.7684940079638206), ('arms', 0.

In [44]:
sim_test(['male', 'spouse'], 'husband')

male, spouse -> husband
GPT-3:
0.9475313396332239
------------------------------------------------------
OPT:
0.6539461379697912
------------------------------------------------------
T5:
0.7028997578998908
------------------------------------------------------


In [45]:
sim_test(['male', 'spouses'], 'husbands')

male, spouses -> husbands
GPT-3:
0.9427816858036653
------------------------------------------------------
OPT:
0.685926507865421
------------------------------------------------------
T5:
0.5320250988109181
------------------------------------------------------


In [46]:
sim_test(['man', 'spouse'], 'husband')

man, spouse -> husband
GPT-3:
0.9496874597051836
------------------------------------------------------
OPT:
0.6912702391360005
------------------------------------------------------
T5:
0.8448111636091383
------------------------------------------------------


In [47]:
sim_test(['men', 'spouses'], 'husbands')

men, spouses -> husbands
GPT-3:
0.9502174456654946
------------------------------------------------------
OPT:
0.7359921769477056
------------------------------------------------------
T5:
0.7571537894770407
------------------------------------------------------


### B - Brothers, sisters, and daughters
Family words related to children.

* Male, sibling, brother.

In [48]:
define(['man', 'sibling'])

man, sibling
GPT-3:
[('man', 0.9573662782650898), ('sibling', 0.9573662770783836), ('brother', 0.9538425559346113), ('person', 0.9347231068272143), ('son', 0.9346143206523212), ('siblings', 0.9335061869293211), ('him', 0.9306810103625845), ('brothers', 0.9303117262337444), ('human', 0.928581883454251), ('you', 0.9276267681419719), ('sum', 0.9270019403201195), ('father', 0.9268902569625375)]
------------------------------------------------------
OPT:
[('man', 0.8941992083753505), ('manner', 0.851686937977097), ('manifest', 0.8498415091924564), ('manuscript', 0.8039399428131107), ('manage', 0.8037715939882866), ('manipulation', 0.7879926790175358), ('manipulate', 0.7583808915679665), ('siblings', 0.7428458467877597), ('sibling', 0.7268027234415497), ('suspect', 0.725408006755357), ('sister', 0.7188946097566331), ('boyfriend', 0.7121058877946541)]
------------------------------------------------------
T5:
[('man', 0.8804988788068665), ('person', 0.7649095639552994), ('woman', 0.7630939440

In [49]:
sim_test(['man', 'sibling'], 'brother')

man, sibling -> brother
GPT-3:
0.9538425559346113
------------------------------------------------------
OPT:
0.6679592841726274
------------------------------------------------------
T5:
0.6809753929994926
------------------------------------------------------


In [50]:
sim_test(['men', 'siblings'], 'brothers')

men, siblings -> brothers
GPT-3:
0.9507396126173475
------------------------------------------------------
OPT:
0.6497953959006818
------------------------------------------------------
T5:
0.4651496301025052
------------------------------------------------------


In [51]:
define(['boys', 'siblings'])

boys, siblings
GPT-3:
[('boys', 0.968080190192264), ('siblings', 0.9680801895977976), ('brothers', 0.9611713297118251), ('sibling', 0.9448565655612772), ('brother', 0.9429287819891876), ('children', 0.9333956376419), ('boy', 0.9258704273176603), ('sisters', 0.923539076000806), ('family', 0.9167276049799848), ('teens', 0.9126933459395592), ('girls', 0.9120033230050758), ('they', 0.9074711223363339)]
------------------------------------------------------
OPT:
[('boys', 0.9144994939179069), ('siblings', 0.8706582398889535), ('girls', 0.8395861740698718), ('sisters', 0.8247531872082055), ('boyfriend', 0.7480487319587138), ('brothers', 0.7373118781751464), ('children', 0.7250318055332488), ('wives', 0.7217291294532366), ('husbands', 0.7206251499438823), ('sometime', 0.7077753150837706), ('sister', 0.7067407336420318), ('daughters', 0.7042429418992611)]
------------------------------------------------------
T5:
[('boys', 0.9073906531223963), ('siblings', 0.8932794121414583), ('boy', 0.760350

In [52]:
sim_test(['boy', 'sibling'], 'brother')

boy, sibling -> brother
GPT-3:
0.9566508382237244
------------------------------------------------------
OPT:
0.6601419070564094
------------------------------------------------------
T5:
0.6939939170037881
------------------------------------------------------


In [53]:
sim_test(['boys', 'siblings'], 'brothers')

boys, siblings -> brothers
GPT-3:
0.9611713297118251
------------------------------------------------------
OPT:
0.7373118781751464
------------------------------------------------------
T5:
0.4459633838939217
------------------------------------------------------


In [54]:
define(['male', 'sibling'])

male, sibling
GPT-3:
[('male', 0.9612182788942357), ('sibling', 0.9612182780753862), ('brother', 0.9478799468180388), ('siblings', 0.9398927757177307), ('brothers', 0.9322481096293992), ('gender', 0.9267300048126865), ('sister', 0.9243294190465867), ('son', 0.923130031331001), ('father', 0.9215559164206671), ('name', 0.9202169095758258), ('human', 0.9185746448511343), ('female', 0.9185088692026457)]
------------------------------------------------------
OPT:
[('male', 0.8719845533989214), ('female', 0.7951852137356272), ('sibling', 0.7514147799072636), ('siblings', 0.7050779337174915), ('sister', 0.6732514443077753), ('suspect', 0.6713579501679824), ('sisters', 0.6662066310651582), ('lesbian', 0.6406996030535843), ('silly', 0.6405271918141169), ('sometime', 0.6338078159615725), ('sadly', 0.6284851268867193), ('sensible', 0.6282936513068036)]
------------------------------------------------------
T5:
[('sibling', 0.8052782460260731), ('male', 0.7672658986331969), ('active', 0.6423306972

In [55]:
sim_test(['male', 'sibling'], 'brother')

male, sibling -> brother
GPT-3:
0.9478799468180388
------------------------------------------------------
OPT:
0.5604806113866312
------------------------------------------------------
T5:
0.48306048355691833
------------------------------------------------------


In [56]:
sim_test(['male', 'siblings'], 'brothers')

male, siblings -> brothers
GPT-3:
0.9429295025476534
------------------------------------------------------
OPT:
0.6768129424215222
------------------------------------------------------
T5:
0.575975806929338
------------------------------------------------------


* Female, sibling, sister

In [57]:
define(['female', 'sibling'])

female, sibling
GPT-3:
[('female', 0.964887082218451), ('sibling', 0.964887081033464), ('sister', 0.956936532291368), ('siblings', 0.9451279462614018), ('sisters', 0.9382796236972685), ('brother', 0.9290062437797563), ('gender', 0.9282455319375151), ('daughter', 0.9261136272829396), ('she', 0.9248209659036907), ('family', 0.9239645695494294), ('girl', 0.9231883889006374), ('mother', 0.9229718334320807)]
------------------------------------------------------
OPT:
[('female', 0.8696045077568799), ('male', 0.8372985080823494), ('sibling', 0.7229800878253165), ('siblings', 0.6856967103113658), ('sister', 0.6618879203167848), ('youngster', 0.6505109408284765), ('sisters', 0.6465228603250284), ('lesbian', 0.6431180149693336), ('silly', 0.6421215123937166), ('suspect', 0.6380382167563743), ('sadly', 0.6345134049908934), ('foreigner', 0.6261013167175813)]
------------------------------------------------------
T5:
[('female', 0.8586028572826583), ('sibling', 0.7602449340258428), ('lady', 0.7177

In [58]:
define(['female', 'siblings'])

female, siblings
GPT-3:
[('female', 0.9622297205723549), ('siblings', 0.9622297178143397), ('sisters', 0.9534547034794214), ('sibling', 0.9530601423356931), ('sister', 0.951755276521169), ('family', 0.9329862795208205), ('daughters', 0.932109676294921), ('couple', 0.9275009377329169), ('gender', 0.924877409346208), ('mother', 0.9242968227078404), ('brother', 0.9231534454879253), ('daughter', 0.922408025139366)]
------------------------------------------------------
OPT:
[('female', 0.8499515266097886), ('male', 0.8177872672584631), ('siblings', 0.7787026433679691), ('sisters', 0.7099030217016422), ('daughter', 0.7055796050116034), ('girls', 0.6888966168355036), ('youngster', 0.6859696240395714), ('boyfriend', 0.6858264685439205), ('brothers', 0.6847739208352482), ('sadly', 0.6773859083342159), ('sibling', 0.6750882649447635), ('herself', 0.6721058709569063)]
------------------------------------------------------
T5:
[('siblings', 0.8942914209405428), ('female', 0.8747180161593393), ('s

In [59]:
sim_test(['female', 'sibling'], 'sister')

female, sibling -> sister
GPT-3:
0.956936532291368
------------------------------------------------------
OPT:
0.6618879203167848
------------------------------------------------------
T5:
0.6980081900570132
------------------------------------------------------


In [60]:
sim_test(['female', 'siblings'], 'sisters')

female, siblings -> sisters
GPT-3:
0.9534547034794214
------------------------------------------------------
OPT:
0.7099030217016422
------------------------------------------------------
T5:
0.4265280378763746
------------------------------------------------------


In [61]:
define(['woman', 'sibling'])

woman, sibling
GPT-3:
[('woman', 0.9562168078353658), ('sibling', 0.9562168077089461), ('sister', 0.9543894620766744), ('female', 0.944884999830079), ('she', 0.9384035070686124), ('mother', 0.9349334016365657), ('siblings', 0.9344449467503059), ('person', 0.9308601648886747), ('family', 0.9267309702592055), ('daughter', 0.9263824643135565), ('wife', 0.9259257874888888), ('name', 0.9249448859425277)]
------------------------------------------------------
OPT:
[('woman', 0.8958763030025035), ('boyfriend', 0.7584730878497212), ('siblings', 0.7576682801851895), ('daughter', 0.7573547102196272), ('girl', 0.7479769177044813), ('sibling', 0.7232207947677805), ('husband', 0.7106162721473408), ('sisters', 0.7062485701845543), ('sister', 0.7058624718911608), ('girlfriend', 0.7051111502295615), ('manuscript', 0.7043571973832932), ('manner', 0.7004203690211133)]
------------------------------------------------------
T5:
[('woman', 0.891207215700693), ('sibling', 0.7853631993800381), ('lady', 0.756

In [62]:
define(['women', 'siblings'])

women, siblings
GPT-3:
[('siblings', 0.9619979945075989), ('sisters', 0.9599318755112884), ('sister', 0.9432754459692267), ('sibling', 0.9413745631830571), ('wives', 0.940524318925006), ('daughters', 0.939222904991416), ('family', 0.9373784123568716), ('female', 0.9363404073457315), ('couple', 0.9295472009544037), ('mother', 0.9251329542577214), ('girls', 0.9224008122922516), ('they', 0.9219652680137902)]
------------------------------------------------------
OPT:
[('siblings', 0.8298052352349449), ('girls', 0.8135327652560369), ('children', 0.8004344606539677), ('boys', 0.7607111822446999), ('wives', 0.754630948575732), ('sisters', 0.7535171269794181), ('woman', 0.7243996525257794), ('daughter', 0.720206164736396), ('husbands', 0.7161681132128597), ('people', 0.6991381676260968), ('boyfriend', 0.6924575700787523), ('spouses', 0.6918388261203707)]
------------------------------------------------------
T5:
[('siblings', 0.8901336925604141), ('sister', 0.7387278719395665), ('daughters', 

In [63]:
sim_test(['woman', 'sibling'], 'sister')

woman, sibling -> sister
GPT-3:
0.9543894620766744
------------------------------------------------------
OPT:
0.7058624718911608
------------------------------------------------------
T5:
0.6800730624736109
------------------------------------------------------


In [64]:
sim_test(['women', 'siblings'], 'sisters')

women, siblings -> sisters
GPT-3:
0.9599318755112884
------------------------------------------------------
OPT:
0.7535171269794181
------------------------------------------------------
T5:
0.6404114406172997
------------------------------------------------------


In [65]:
define(['girl', 'sibling'])

girl, sibling
GPT-3:
[('sibling', 0.9628090714279097), ('girl', 0.9628090711754012), ('sister', 0.9510075633557207), ('siblings', 0.9394049758598502), ('girls', 0.9342970357455234), ('female', 0.9293413919504111), ('sisters', 0.9282140637681622), ('daughter', 0.927431616367755), ('brother', 0.9261779259254909), ('she', 0.925852188974991), ('child', 0.924836644897143), ('kid', 0.9206238275706947)]
------------------------------------------------------
OPT:
[('girl', 0.8891954581372763), ('boyfriend', 0.7888656153245834), ('sibling', 0.7615359350726484), ('siblings', 0.759710049950925), ('sister', 0.7565864825285684), ('boy', 0.7464539570267237), ('sisters', 0.7389243647844022), ('boys', 0.7305695617249894), ('girls', 0.7168829176652383), ('guy', 0.71393722609625), ('woman', 0.7107001788609899), ('daughter', 0.7090099845882787)]
------------------------------------------------------
T5:
[('girl', 0.8936250389387532), ('child', 0.7721686339085962), ('woman', 0.7675035896219979), ('sibling

In [66]:
define(['girl', 'siblings'])

girl, siblings
GPT-3:
[('girl', 0.9583381030397391), ('siblings', 0.9583381016856872), ('sibling', 0.9527504112494598), ('girls', 0.9490887353743767), ('sister', 0.9475885608757655), ('sisters', 0.9451794541147139), ('female', 0.9283334064237518), ('daughter', 0.9254697386302507), ('child', 0.9243535324049073), ('children', 0.9237093756167394), ('brother', 0.9220462628667201), ('she', 0.9217013368789512)]
------------------------------------------------------
OPT:
[('girl', 0.8852668023139248), ('siblings', 0.8280236059086299), ('boyfriend', 0.8244876625043984), ('boys', 0.7879193350336722), ('girls', 0.7797924247550605), ('sisters', 0.7782721232883294), ('daughter', 0.7667879952091907), ('boy', 0.7501939017752398), ('sister', 0.7434614046810059), ('woman', 0.7418679750988437), ('girlfriend', 0.7392181606907489), ('brothers', 0.73762836536465)]
------------------------------------------------------
T5:
[('girl', 0.8908435183584397), ('siblings', 0.874032446422924), ('boys', 0.805054469

In [67]:
sim_test(['girl', 'sibling'], 'sister')

girl, sibling -> sister
GPT-3:
0.9510075633557207
------------------------------------------------------
OPT:
0.7565864825285684
------------------------------------------------------
T5:
0.692912661643289
------------------------------------------------------


In [68]:
sim_test(['girls', 'siblings'], 'sisters')

girls, siblings -> sisters
GPT-3:
0.9599722957779924
------------------------------------------------------
OPT:
0.8193700584900423
------------------------------------------------------
T5:
0.6547863397406923
------------------------------------------------------


* Female, children, daughter

In [69]:
define(['girl', 'children'])

girl, children
GPT-3:
[('children', 0.9666316410283714), ('girl', 0.9666316396109573), ('child', 0.9578169941510852), ('girls', 0.9569423579634053), ('kid', 0.9365856002011393), ('female', 0.9236825190408627), ('baby', 0.9213930709488336), ('boy', 0.9209232812562047), ('daughter', 0.9199574343996691), ('gender', 0.913294091740371), ('she', 0.9126891563825096), ('around', 0.9118148010487241)]
------------------------------------------------------
OPT:
[('children', 0.9053640669466504), ('girl', 0.8778833794471848), ('girls', 0.8251191996684122), ('boyfriend', 0.8250804372135493), ('boys', 0.814604193565465), ('daughter', 0.8110034651073659), ('woman', 0.7987131979337789), ('girlfriend', 0.7501947455845457), ('boy', 0.7380573775247147), ('guy', 0.7102404416086349), ('brothers', 0.7064979339185524), ('himself', 0.6864513418809611)]
------------------------------------------------------
T5:
[('girl', 0.9033216759333804), ('child', 0.7944891108769521), ('woman', 0.7841803155316929), ('teach

In [70]:
define(['woman', 'children'])

woman, children
GPT-3:
[('children', 0.957642681126684), ('woman', 0.9576426797294372), ('child', 0.9515767830687498), ('female', 0.9414757326639832), ('mother', 0.9357109423368348), ('human', 0.9285243405290786), ('she', 0.9274153649989352), ('girl', 0.9271802905074749), ('baby', 0.9241854350831853), ('person', 0.923791908376834), ('people', 0.9233478156446084), ('daughter', 0.9211607747746374)]
------------------------------------------------------
OPT:
[('children', 0.913372382690083), ('woman', 0.9082524255337142), ('daughter', 0.8188720872690616), ('boyfriend', 0.7766664384562973), ('girl', 0.7477719081366265), ('girls', 0.747217379888995), ('girlfriend', 0.7354828996571111), ('boys', 0.7302004975179266), ('sheep', 0.7116746386052021), ('himself', 0.7001488257379618), ('people', 0.6982470111105588), ('manuscript', 0.6969570680612884)]
------------------------------------------------------
T5:
[('woman', 0.9046562330251027), ('children', 0.7950145448191855), ('person', 0.7514659448

In [71]:
define(['female', 'children'])

female, children
GPT-3:
[('children', 0.9638928917024139), ('female', 0.9638928916306357), ('child', 0.9527661609897715), ('girls', 0.9368583525631906), ('girl', 0.9317922956893646), ('gender', 0.9289898622560189), ('baby', 0.927764937723071), ('mother', 0.9260793482843631), ('daughter', 0.9232514041705409), ('woman', 0.922910988319164), ('around', 0.9201858770467675), ('human', 0.9199507596804941)]
------------------------------------------------------
OPT:
[('children', 0.8799609994360587), ('female', 0.8401436416326484), ('male', 0.8012360839969532), ('daughter', 0.7587765194909271), ('girls', 0.7486668693776524), ('woman', 0.722630964934138), ('youngster', 0.71279625477869), ('boys', 0.7112860404487247), ('boyfriend', 0.7075633678329626), ('ourselves', 0.7073289805859501), ('themselves', 0.7052157425537731), ('herself', 0.6893832831595582)]
------------------------------------------------------
T5:
[('female', 0.8646245181214878), ('children', 0.7476650419415389), ('woman', 0.73542

In [72]:
sim_test(['girl', 'children'], 'daughter')

girl, children -> daughter
GPT-3:
0.9199574343996691
------------------------------------------------------
OPT:
0.8110034651073659
------------------------------------------------------
T5:
0.6570462354136002
------------------------------------------------------


In [73]:
sim_test(['woman', 'children'], 'daughter')

woman, children -> daughter
GPT-3:
0.9211607747746374
------------------------------------------------------
OPT:
0.8188720872690616
------------------------------------------------------
T5:
0.6018252137782943
------------------------------------------------------


In [74]:
sim_test(['female', 'children'], 'daughter')

female, children -> daughter
GPT-3:
0.9232514041705409
------------------------------------------------------
OPT:
0.7587765194909271
------------------------------------------------------
T5:
0.6139559882019199
------------------------------------------------------


In [75]:
define(['female', 'offspring'])

female, offspring
GPT-3:
[('female', 0.9617593386910952), ('offspring', 0.9617593339015695), ('mother', 0.9245037755057686), ('gender', 0.9229292130021383), ('birth', 0.9225665466305328), ('baby', 0.9220181296016174), ('daughter', 0.9184087667062377), ('sister', 0.9163182226148394), ('pregnant', 0.9158022556661689), ('pregnancy', 0.9148222167086872), ('daughters', 0.9132635458354439), ('woman', 0.9130242113229747)]
------------------------------------------------------
OPT:
[('female', 0.8769243737971986), ('male', 0.8406021341719185), ('offspring', 0.8110480389199101), ('youngster', 0.71397201377457), ('offence', 0.7051981837937658), ('offense', 0.7004268606032557), ('offender', 0.6960786842775357), ('yourself', 0.6897723616817422), ('engineer', 0.6889762449984507), ('herself', 0.6880334124183309), ('foreigner', 0.6852204350086004), ('offsprings', 0.6803824325122294)]
------------------------------------------------------
T5:
[('offspring', 0.8272454102135459), ('female', 0.8225446480

In [76]:
define(['female', 'offsprings'])

female, offsprings
GPT-3:
[('female', 0.9566181267454683), ('offsprings', 0.9566181255520363), ('gender', 0.9208111712737939), ('woman', 0.9180429001943269), ('there', 0.9164066635673761), ('feat', 0.9155374657535539), ('upside', 0.9144139582381805), ('she', 0.9143867855900403), ('offspring', 0.9142626523138018), ('around', 0.9118652489696568), ('favour', 0.9115125734769677), ('character', 0.9097998756791102)]
------------------------------------------------------
OPT:
[('female', 0.8772582372879161), ('male', 0.8370685412484005), ('offspring', 0.7531183595094475), ('offsprings', 0.7402139265515815), ('youngster', 0.6830849579913467), ('yourself', 0.6784673821278959), ('herself', 0.6731134827492391), ('offence', 0.6699710875778577), ('engineer', 0.6676104471682449), ('offense', 0.6669715456644898), ('supportive', 0.6617256093925503), ('straightforward', 0.6610580412400582)]
------------------------------------------------------
T5:
[('female', 0.8447598044448621), ('offsprings', 0.7707

In [77]:
sim_test(['female', 'offspring'], 'daughter')

female, offspring -> daughter
GPT-3:
0.9184087667062377
------------------------------------------------------
OPT:
0.6788833297252161
------------------------------------------------------
T5:
0.7008428263911035
------------------------------------------------------


In [78]:
sim_test(['female', 'offsprings'], 'daughters')

female, offsprings -> daughters
GPT-3:
0.8889128974904628
------------------------------------------------------
OPT:
0.5291346478567805
------------------------------------------------------
T5:
0.5803639840018378
------------------------------------------------------


In [79]:
define(['woman', 'offspring'])

woman, offspring
GPT-3:
[('woman', 0.9516310860223764), ('offspring', 0.9516310814353685), ('female', 0.9431058687854854), ('mother', 0.9379533591020421), ('birth', 0.9255613899788676), ('pregnancy', 0.9229231222942266), ('pregnant', 0.9222948567290469), ('baby', 0.9221356339607663), ('she', 0.9204704371650523), ('daughter', 0.9200157795940154), ('person', 0.9190676130286912), ('wife', 0.9187500914861754)]
------------------------------------------------------
OPT:
[('woman', 0.9104500575109127), ('offspring', 0.8290045102749302), ('daughter', 0.7846604236132395), ('boyfriend', 0.7802775058304452), ('girl', 0.7464745356607179), ('manuscript', 0.7420490542659202), ('sheep', 0.7340152699860227), ('momentum', 0.7329296141695185), ('girlfriend', 0.7327031348155478), ('manipulation', 0.73173477833268), ('manner', 0.7308870343913914), ('children', 0.7296359741056448)]
------------------------------------------------------
T5:
[('woman', 0.8352342958176003), ('offspring', 0.8109926276676026),

In [80]:
define(['woman', 'offsprings'])

woman, offsprings
GPT-3:
[('offsprings', 0.9516872599172511), ('woman', 0.9516872597477257), ('female', 0.9326867287420209), ('she', 0.9243460677763029), ('person', 0.9185539059416321), ('mother', 0.9163916371967352), ('there', 0.9135744541701708), ('upside', 0.913292680929158), ('human', 0.9118822696383855), ('character', 0.9100892780419656), ('gender', 0.9098402065036402), ('round', 0.9090756255173522)]
------------------------------------------------------
OPT:
[('woman', 0.9082012586585125), ('offspring', 0.7803676534173442), ('boyfriend', 0.7601899909483891), ('daughter', 0.7601110805149499), ('offsprings', 0.7588610343422737), ('girl', 0.7354418825263992), ('manipulation', 0.7313596702673792), ('manuscript', 0.7305601752874594), ('manner', 0.7216257628257177), ('momentum', 0.7196760437900349), ('girlfriend', 0.7121228157695152), ('sheep', 0.7104152890763034)]
------------------------------------------------------
T5:
[('woman', 0.8521660358380811), ('offsprings', 0.74149050047880

In [81]:
sim_test(['woman', 'offspring'], 'daughter')

woman, offspring -> daughter
GPT-3:
0.9200157795940154
------------------------------------------------------
OPT:
0.7846604236132395
------------------------------------------------------
T5:
0.7169954828922823
------------------------------------------------------


In [82]:
sim_test(['women', 'offsprings'], 'daughters')

women, offsprings -> daughters
GPT-3:
0.8964918869907845
------------------------------------------------------
OPT:
0.6137859137416883
------------------------------------------------------
T5:
0.5420116275092903
------------------------------------------------------


### C - Animals, diminutives, and augmentatives!
Is a baby duck a duckling?

In [83]:
define(['baby', 'duck'])

baby, duck
GPT-3:
[('baby', 0.9630496707306292), ('duck', 0.9630496693485963), ('duckling', 0.9483280181669149), ('child', 0.9249847872533217), ('infant', 0.924552270748521), ('birth', 0.9244885536809033), ('there', 0.9204207683220613), ('down', 0.9191436793566761), ('rid', 0.9173116624512383), ('buck', 0.9173000798493369), ('egg', 0.9172971903822483), ('puppy', 0.9154902777645133)]
------------------------------------------------------
OPT:
[('baby', 0.894463465190662), ('duckling', 0.8207083691540811), ('duck', 0.7332403900324568), ('dull', 0.6367049743060318), ('kidney', 0.6322666215893379), ('boyfriend', 0.6314013484135944), ('dog', 0.6271043618752745), ('duo', 0.6156070181484683), ('grandchild', 0.6055716003551901), ('brother', 0.6046235959348372), ('youngster', 0.599783569491756), ('manner', 0.5986339110729503)]
------------------------------------------------------
T5:
[('duck', 0.8111561428227232), ('baby', 0.7458023491115191), ('box', 0.6649496712444745), ('cupboard', 0.640738

In [84]:
sim_test(['baby', 'duck'], 'duckling')

baby, duck -> duckling
GPT-3:
0.9483280181669149
------------------------------------------------------
OPT:
0.8207083691540811
------------------------------------------------------
T5:
0.5437253039963204
------------------------------------------------------


In [85]:
define(['small', 'duck'])

small, duck
GPT-3:
[('duck', 0.9543271657518128), ('small', 0.9543271626389949), ('down', 0.9320061022676656), ('duckling', 0.9271576660268376), ('tiny', 0.9267853939058065), ('low', 0.9214774760050461), ('there', 0.9210601246884299), ('under', 0.9199454199913306), ('rid', 0.9169712462895157), ('close', 0.9164451871271608), ('round', 0.9147054674391402), ('list', 0.9140228197673019)]
------------------------------------------------------
OPT:
[('small', 0.8804568409421278), ('tiny', 0.7546449621021287), ('large', 0.7411797068168522), ('duckling', 0.7401965527093617), ('large-scale', 0.6976518352442993), ('duck', 0.6783245829968988), ('dull', 0.6762745283195434), ('big', 0.6621896810968525), ('medium', 0.6452811926645569), ('huge', 0.6409633443508636), ('largely', 0.6406186402187972), ('massive', 0.6255351577277019)]
------------------------------------------------------
T5:
[('duck', 0.884120588913419), ('small', 0.859916244928304), ('tiny', 0.7831339706434157), ('bright', 0.7571820101

In [86]:
sim_test(['small', 'duck'], 'duckling')

small, duck -> duckling
GPT-3:
0.9271576660268376
------------------------------------------------------
OPT:
0.7401965527093617
------------------------------------------------------
T5:
0.6661620265520534
------------------------------------------------------


In [87]:
define(['little', 'duck'])

little, duck
GPT-3:
[('little', 0.9521555373314179), ('duck', 0.9521555363689829), ('duckling', 0.9395635829634439), ('down', 0.9279849477068647), ('small', 0.9202107087210366), ('tiny', 0.916057438976834), ('under', 0.9155946329380066), ('there', 0.9146095198153161), ('child', 0.9117369602096537), ('bottom', 0.9111347295486614), ('how', 0.9092549796062958), ('baby', 0.9089653964032308)]
------------------------------------------------------
OPT:
[('little', 0.8787631575597559), ('duckling', 0.729711828186326), ('duck', 0.6831783235210029), ('tiny', 0.6809824543845104), ('dull', 0.6374447970144536), ('small', 0.5929536173148897), ('duo', 0.5858706356484976), ('big', 0.585502698709921), ('dual', 0.5760246230614768), ('cheerful', 0.5613517980893158), ('youngster', 0.5467270868598595), ('supermarket', 0.5446819952661517)]
------------------------------------------------------
T5:
[('little', 0.874713191121589), ('duck', 0.8692010332432655), ('tiny', 0.7915835197535177), ('eager', 0.767763

In [88]:
sim_test(['little', 'duck'], 'duckling')

little, duck -> duckling
GPT-3:
0.9395635829634439
------------------------------------------------------
OPT:
0.729711828186326
------------------------------------------------------
T5:
0.663642908682794
------------------------------------------------------


In [89]:
define(['child', 'duck'])

child, duck
GPT-3:
[('child', 0.959754151486522), ('duck', 0.9597541496963414), ('duckling', 0.9485760597022014), ('children', 0.937577897269021), ('baby', 0.9347632734939539), ('kid', 0.926786706823965), ('there', 0.9245866126576519), ('down', 0.9240708844710344), ('rid', 0.9202206251274159), ('how', 0.9188377608548518), ('boy', 0.917994337957256), ('bird', 0.917424905347453)]
------------------------------------------------------
OPT:
[('child', 0.8811302382355695), ('duckling', 0.765976836763266), ('duck', 0.7302206881660855), ('grandchild', 0.6646453329599453), ('kidney', 0.6604081791855867), ('dog', 0.644023947462187), ('kid', 0.628099113653354), ('kidnap', 0.6142059816594493), ('dull', 0.6141397437151076), ('children', 0.6009777261839939), ('horse', 0.6006685471427268), ('bird', 0.595389977089855)]
------------------------------------------------------
T5:
[('child', 0.8959581469187533), ('duck', 0.8380033188288087), ('insect', 0.7968319674066293), ('tree', 0.7941288855727333), (

In [90]:
sim_test(['child', 'duck'], 'duckling')

child, duck -> duckling
GPT-3:
0.9485760597022014
------------------------------------------------------
OPT:
0.765976836763266
------------------------------------------------------
T5:
0.6190045792536467
------------------------------------------------------


In [91]:
define(['young', 'duck'])

young, duck
GPT-3:
[('young', 0.9557138663913897), ('duck', 0.9557138648708687), ('duckling', 0.9294376230205035), ('there', 0.923915077911029), ('rid', 0.9213392419600231), ('you', 0.9210674542288999), ('youth', 0.9210273807674823), ('under', 0.9206702091773388), ('one', 0.9193329104365233), ('opt', 0.9190742308202825), ('how', 0.9189168125250601), ('youngster', 0.9187949855648048)]
------------------------------------------------------
OPT:
[('young', 0.8914778051896193), ('youngster', 0.8417356613374702), ('duckling', 0.759709427126056), ('duck', 0.698462657537282), ('dull', 0.6773262238856027), ('cheerful', 0.6433441051564903), ('drunk', 0.6347669876309893), ('duo', 0.6303994978374998), ('gay', 0.6237278171890841), ('lesbian', 0.6221768757498223), ('tall', 0.6209047051740483), ('old-fashioned', 0.6170904121288536)]
------------------------------------------------------
T5:
[('young', 0.8899399734808556), ('duck', 0.8732044960051137), ('eager', 0.797090838722169), ('cheerful', 0.785

In [92]:
sim_test(['young', 'duck'], 'duckling')

young, duck -> duckling
GPT-3:
0.9294376230205035
------------------------------------------------------
OPT:
0.759709427126056
------------------------------------------------------
T5:
0.6343379468763657
------------------------------------------------------


In [93]:
define(['small', 'cat'])

small, cat
GPT-3:
[('cat', 0.9547398196171303), ('small', 0.9547398156013137), ('pet', 0.9271971209094247), ('down', 0.9214190742630344), ('tiny', 0.9196981960753107), ('dog', 0.918244191940466), ('there', 0.9177239607990495), ('close', 0.9161864037336761), ('low', 0.9152816573858107), ('kitten', 0.9144343215546878), ('one', 0.913943789594781), ('under', 0.9136109791889321)]
------------------------------------------------------
OPT:
[('small', 0.8209442054490963), ('cat', 0.7990171249809421), ('catalogue', 0.7554134099789742), ('tiny', 0.7208996460281327), ('large', 0.7150408502558051), ('large-scale', 0.6799754208278088), ('medium', 0.636466290043039), ('largely', 0.6348095243007607), ('dog', 0.6318710044763178), ('big', 0.6271790490101511), ('horse', 0.6240998413132238), ('ratio', 0.622279324032994)]
------------------------------------------------------
T5:
[('cat', 0.8907750128772927), ('small', 0.8431393806047112), ('tiny', 0.7769826596378606), ('cheap', 0.7644953660924987), ('bl

In [94]:
sim_test(['small', 'cat'], 'kitten')

small, cat -> kitten
GPT-3:
0.9144343215546878
------------------------------------------------------
OPT:
0.5512440564332646
------------------------------------------------------
T5:
0.40799627401039307
------------------------------------------------------


In [95]:
define(['baby', 'cat'])

baby, cat
GPT-3:
[('baby', 0.9627088624346655), ('cat', 0.9627088618041234), ('kitten', 0.941256265442302), ('pet', 0.9353502278711769), ('child', 0.9308834499165959), ('dog', 0.9266288592304173), ('infant', 0.9227496752331122), ('puppy', 0.9213753613528664), ('birth', 0.9198636149257411), ('there', 0.9178328590655713), ('animal', 0.9150242918376973), ('rat', 0.9148801643131865)]
------------------------------------------------------
OPT:
[('baby', 0.862008353638249), ('cat', 0.8518424458983207), ('catalogue', 0.7794657930588615), ('dog', 0.70858555302566), ('horse', 0.672169256401049), ('duckling', 0.6525414299280246), ('kitten', 0.6465518780434741), ('kidney', 0.6426282651023386), ('boyfriend', 0.6374430387148684), ('bird', 0.6326286276592845), ('painter', 0.6170127458221375), ('grandchild', 0.6148238791120569)]
------------------------------------------------------
T5:
[('cat', 0.8383279275464726), ('baby', 0.7400089353758057), ('hair', 0.6786094918082202), ('box', 0.676339657677519

In [96]:
sim_test(['baby', 'cat'], 'kitten')

baby, cat -> kitten
GPT-3:
0.941256265442302
------------------------------------------------------
OPT:
0.6465518780434741
------------------------------------------------------
T5:
0.4958669884472512
------------------------------------------------------


In [97]:
define(['child', 'cat'])

child, cat
GPT-3:
[('child', 0.9625442760647225), ('cat', 0.9625442751690445), ('kitten', 0.9374190578513657), ('pet', 0.9364259471550237), ('baby', 0.9313718239227277), ('kid', 0.9302270224920524), ('children', 0.929336687894052), ('dog', 0.9289953849179579), ('rat', 0.9204654370917789), ('there', 0.9189922704066238), ('animal', 0.9175600755023097), ('mouse', 0.9140432671475167)]
------------------------------------------------------
OPT:
[('cat', 0.8624346671482463), ('child', 0.8568505761241643), ('catalogue', 0.764858985916628), ('dog', 0.7157417412367748), ('horse', 0.6712778957523382), ('kidney', 0.6570582896214938), ('grandchild', 0.6542941881192954), ('bird', 0.6461588789827313), ('animal', 0.640457415773076), ('kid', 0.633156968380381), ('kitten', 0.6205629348349764), ('children', 0.6060572291444672)]
------------------------------------------------------
T5:
[('child', 0.9257443004787131), ('cat', 0.9033313771518565), ('animal', 0.8367537708087454), ('car', 0.8236974248161256

In [98]:
sim_test(['child', 'cat'], 'kitten')

child, cat -> kitten
GPT-3:
0.9374190578513657
------------------------------------------------------
OPT:
0.6205629348349764
------------------------------------------------------
T5:
0.34159632272381235
------------------------------------------------------


In [99]:
define(['young', 'cat'])

young, cat
GPT-3:
[('young', 0.9537116545407746), ('cat', 0.9537116539033017), ('one', 0.9247576997631406), ('pet', 0.9236711477759256), ('there', 0.9229134999873934), ('you', 0.9228959491630201), ('dog', 0.9211000513016342), ('what', 0.9188147186394754), ('kitten', 0.9171810610076653), ('two', 0.9171239572054586), ('opt', 0.9168443050819362), ('under', 0.9166598275357576)]
------------------------------------------------------
OPT:
[('young', 0.8258651991655515), ('youngster', 0.8142098623795668), ('cat', 0.7960536857347205), ('catalogue', 0.7447181373878788), ('dog', 0.658688762270336), ('horse', 0.6438308878593804), ('cheerful', 0.6341159204996762), ('drunk', 0.6326015002244921), ('tall', 0.6299397468659222), ('duckling', 0.6283148220729475), ('grandson', 0.6269489850943691), ('gay', 0.624879683445964)]
------------------------------------------------------
T5:
[('cat', 0.8784458922025794), ('young', 0.874879766753941), ('black', 0.7897581702889697), ('hair', 0.7776120605232377), ('

In [100]:
sim_test(['young', 'cat'], 'kitten')

young, cat -> kitten
GPT-3:
0.9171810610076653
------------------------------------------------------
OPT:
0.6001082578605921
------------------------------------------------------
T5:
0.4055802314532067
------------------------------------------------------


In [101]:
define(['small', 'dog'])

small, dog
GPT-3:
[('small', 0.9583667806901863), ('dog', 0.9583667802115712), ('down', 0.9256843276414166), ('pet', 0.9251226205366565), ('tiny', 0.9228784853517158), ('puppy', 0.9212354826175428), ('little', 0.9178509869830587), ('low', 0.9167477309511298), ('under', 0.9158510538018783), ('there', 0.9145464816355001), ('close', 0.9141502123388803), ('short', 0.9133057473309605)]
------------------------------------------------------
OPT:
[('small', 0.8294988964857011), ('dog', 0.8073646377028512), ('tiny', 0.7203701827224069), ('large', 0.7174874792946937), ('large-scale', 0.6824330059884973), ('big', 0.6685078849919901), ('largely', 0.650913048318628), ('horse', 0.6505769297198342), ('medium', 0.6433313601793473), ('catalogue', 0.640739914351652), ('badge', 0.6308619632584425), ('duckling', 0.6304422975990865)]
------------------------------------------------------
T5:
[('dog', 0.8905070869026936), ('small', 0.8158415005626314), ('hair', 0.7912096411789592), ('car', 0.78436008493586

In [102]:
sim_test(['small', 'dog'], 'puppy')

small, dog -> puppy
GPT-3:
0.9212354826175428
------------------------------------------------------
OPT:
0.4666601981846175
------------------------------------------------------
T5:
0.3005619832966183
------------------------------------------------------


In [103]:
define(['baby', 'dog'])

baby, dog
GPT-3:
[('baby', 0.966362464587684), ('dog', 0.9663624606687455), ('puppy', 0.9480546427847307), ('child', 0.9371846325564197), ('pet', 0.9332365091074823), ('animal', 0.925424288809845), ('infant', 0.9251291121610514), ('birth', 0.9232912186499038), ('human', 0.9163050087875577), ('cat', 0.9158320941998288), ('breed', 0.9156003243749646), ('kitten', 0.9152303080335964)]
------------------------------------------------------
OPT:
[('baby', 0.8778958179179752), ('dog', 0.8680791995952795), ('horse', 0.6911126224206184), ('boyfriend', 0.6843187639614381), ('duckling', 0.6759419344984132), ('kidney', 0.6694456237310518), ('catalogue', 0.6640439340491644), ('cat', 0.6629835241132473), ('girl', 0.6539010431561243), ('boy', 0.6396276915199143), ('grandchild', 0.6387029059139404), ('sheep', 0.6333864532920828)]
------------------------------------------------------
T5:
[('dog', 0.858494658181624), ('baby', 0.7335289008416532), ('car', 0.7210993805934933), ('tooth', 0.719797577014324

In [104]:
sim_test(['baby', 'dog'], 'puppy')

baby, dog -> puppy
GPT-3:
0.9480546427847307
------------------------------------------------------
OPT:
0.5681283038919827
------------------------------------------------------
T5:
0.4387037841324975
------------------------------------------------------


In [105]:
define(['child', 'dog'])

child, dog
GPT-3:
[('child', 0.9674615177060625), ('dog', 0.9674615136900864), ('puppy', 0.9385687321958838), ('kid', 0.9344722721848178), ('baby', 0.9339231170813105), ('children', 0.9337918462064155), ('pet', 0.9330874644039235), ('animal', 0.9267402202547022), ('human', 0.9209646752927987), ('boy', 0.9186698301811438), ('pupil', 0.9157405613689196), ('there', 0.9146142717664723)]
------------------------------------------------------
OPT:
[('dog', 0.8784206789173092), ('child', 0.8744111417118555), ('horse', 0.6904473087912637), ('kidney', 0.684016717898875), ('grandchild', 0.6778190917680975), ('cat', 0.6689442629130073), ('kid', 0.6674091863482663), ('animal', 0.6560799212574897), ('catalogue', 0.6468955029981814), ('boyfriend', 0.6462156103033386), ('bird', 0.6460141994762281), ('children', 0.6447010072645557)]
------------------------------------------------------
T5:
[('child', 0.9415636226103674), ('dog', 0.9346089820401604), ('animal', 0.8446322316706296), ('car', 0.841001431

In [106]:
sim_test(['child', 'dog'], 'puppy')

child, dog -> puppy
GPT-3:
0.9385687321958838
------------------------------------------------------
OPT:
0.548987329338477
------------------------------------------------------
T5:
0.24878714852967937
------------------------------------------------------


In [107]:
define(['young', 'dog'])

young, dog
GPT-3:
[('young', 0.9592947683578482), ('dog', 0.9592947637283238), ('youngster', 0.9218565070199087), ('one', 0.9213311372583399), ('puppy', 0.9199400163113634), ('pet', 0.919728478310057), ('boy', 0.9196527522699851), ('child', 0.9195249522784168), ('kid', 0.9190865379086983), ('there', 0.9178375117466769), ('under', 0.9170169980037683), ('you', 0.9169401608673413)]
------------------------------------------------------
OPT:
[('young', 0.8461085389120371), ('youngster', 0.8350036726764218), ('dog', 0.8188821752515285), ('boyfriend', 0.6636117159748182), ('grandson', 0.6594188444939832), ('horse', 0.6592891093079946), ('duckling', 0.648391717693082), ('grandmother', 0.6406064534639601), ('baby', 0.6400218102344817), ('granddaughter', 0.6384563953284226), ('gay', 0.6379201829008128), ('cheerful', 0.6367073108006652)]
------------------------------------------------------
T5:
[('dog', 0.89990412390315), ('young', 0.8800777300010468), ('child', 0.800242772261139), ('car', 0.78

In [108]:
sim_test(['young', 'dog'], 'puppy')

young, dog -> puppy
GPT-3:
0.9199400163113634
------------------------------------------------------
OPT:
0.49756465226580016
------------------------------------------------------
T5:
0.29571800800399883
------------------------------------------------------


In [155]:
define(['female', 'fox'])

female, fox
GPT-3:
[('female', 0.9655432474453627), ('she', 0.9332559575534465), ('there', 0.9294121534066991), ('woman', 0.9271558897695149), ('front', 0.9259291835391804), ('opt', 0.9231559042782728), ('what', 0.9224346255471222), ('feat', 0.9223323578257272), ('how', 0.9220060409290064), ('favour', 0.9218210057920708), ('you', 0.9216255274540528), ('human', 0.9215696340465633)]
------------------------------------------------------
OPT:
[('female', 0.8397913363521319), ('male', 0.8071670507796422), ('youngster', 0.6696169070721678), ('foreigner', 0.657271860149731), ('faithful', 0.6527436350783564), ('engineer', 0.6440295401905338), ('theater', 0.6403222538863971), ('fashionable', 0.6377070778109117), ('duckling', 0.6361074559434196), ('firefighter', 0.6348559107090208), ('alliance', 0.63461981842736), ('dozen', 0.6338780984120702)]
------------------------------------------------------
T5:
[('female', 0.8575584387266288), ('lady', 0.7192905971690753), ('pig', 0.7086151761268974), (

In [156]:
sim_test(['female', 'fox'], 'vixen')

female, fox -> vixen
GPT-3:
0.9035699527617026
------------------------------------------------------
OPT:
0.5339479610228397
------------------------------------------------------
T5:
0.5299603844842964
------------------------------------------------------


In [109]:
define(['baby', 'male', 'horse'])

baby, male, horse
GPT-3:
[('horse', 0.9493931249491062), ('baby', 0.9475857677405611), ('male', 0.9467038317898835), ('human', 0.9360320580800465), ('gender', 0.9262456027062288), ('child', 0.9258798510874955), ('there', 0.9256608753260486), ('round', 0.9251667517058652), ('female', 0.9249579490491145), ('birth', 0.9243998586171308), ('name', 0.9238668760377426), ('rider', 0.9237511506340528)]
------------------------------------------------------
OPT:
[('baby', 0.8349455574138903), ('horse', 0.8076683808412926), ('male', 0.8001496468469724), ('female', 0.7637148323364791), ('youngster', 0.7119349865136008), ('boyfriend', 0.6999000389535964), ('girl', 0.6843818518734348), ('firefighter', 0.6705565667413648), ('daughter', 0.6697662564228479), ('duckling', 0.6666828272394811), ('dog', 0.6586722795622303), ('catalogue', 0.6572506937695992)]
------------------------------------------------------
T5:
[('horse', 0.8256853309022314), ('male', 0.8155582495955829), ('purple', 0.7186890361372655

In [110]:
define(['adult', 'female', 'horse'])

adult, female, horse
GPT-3:
[('female', 0.9567790500269334), ('adult', 0.9525388077079895), ('horse', 0.9414787976068394), ('human', 0.9364528926418374), ('gender', 0.9337464890166702), ('there', 0.9308942860704064), ('person', 0.9300616124943157), ('woman', 0.9297322914516913), ('character', 0.9296100017969573), ('name', 0.9281583398544644), ('fare', 0.9275921109611419), ('she', 0.9272388035810675)]
------------------------------------------------------
OPT:
[('female', 0.8428321396536606), ('adult', 0.8175155616585021), ('male', 0.8162504675548637), ('horse', 0.7760184072943985), ('youngster', 0.7222063439613522), ('foreigner', 0.6613574523260123), ('generic', 0.6526464512496507), ('fashionable', 0.6520979545273138), ('theater', 0.6519344712078943), ('realistic', 0.6518423168705352), ('lesbian', 0.6514300371691439), ('ordinary', 0.647941393999564)]
------------------------------------------------------
T5:
[('adult', 0.8521792011851385), ('female', 0.8293410760961477), ('woman', 0.76

In [111]:
define(['adult', 'male', 'pig'])

adult, male, pig
GPT-3:
[('male', 0.9573169459018567), ('adult', 0.9538856645418673), ('pig', 0.9400127051618556), ('human', 0.9374536594707983), ('person', 0.9338801815110107), ('man', 0.9329092293771348), ('sum', 0.9304562357817823), ('you', 0.9303302789113049), ('there', 0.9301534095880066), ('one', 0.9297896797007812), ('body', 0.9294996830286679), ('gender', 0.9287064086364679)]
------------------------------------------------------
OPT:
[('male', 0.8286694089944789), ('adult', 0.8123388025819994), ('female', 0.7736720503835617), ('youngster', 0.6591984521202903), ('lesbian', 0.6465002420914211), ('adolescent', 0.6415345890251316), ('gay', 0.6405904085519405), ('predictable', 0.6318039646964628), ('duckling', 0.6289669912171402), ('passionate', 0.627968563970703), ('gentleman', 0.624790403966791), ('pig', 0.6194265686058273)]
------------------------------------------------------
T5:
[('adult', 0.8714601576938821), ('pig', 0.7844867868965968), ('bird', 0.7655514087310313), ('anima

In [112]:
define(['adult', 'male', 'human'])

adult, male, human
GPT-3:
[('human', 0.9667348422758713), ('male', 0.9644083361442223), ('adult', 0.9600780307853494), ('person', 0.9442980446035549), ('gender', 0.9371474863347216), ('man', 0.9365431782596227), ('name', 0.9300976095847333), ('body', 0.9292559695300757), ('character', 0.9284925587891137), ('female', 0.9279733667113229), ('there', 0.9238265936630139), ('someone', 0.9236884735796738)]
------------------------------------------------------
OPT:
[('male', 0.8530232571540606), ('human', 0.8274489870004416), ('female', 0.8194673254437493), ('adult', 0.8087271299612927), ('humanitarian', 0.7787857266589486), ('humanity', 0.7569356032760375), ('youngster', 0.6714218509142169), ('foreigner', 0.6668161342152669), ('individual', 0.6527066319027248), ('normal', 0.6426561947520104), ('realistic', 0.6417759767636804), ('professional', 0.6412246575799063)]
------------------------------------------------------
T5:
[('adult', 0.8930622003370666), ('human', 0.888335056831248), ('animal

In [113]:
define(['adult', 'female', 'human'])

adult, female, human
GPT-3:
[('human', 0.9663617193441947), ('female', 0.9623084835338672), ('adult', 0.9594009933131717), ('person', 0.9454620595684969), ('gender', 0.9415367935425709), ('woman', 0.9412391469292296), ('name', 0.9330252815533897), ('character', 0.9326333670478031), ('male', 0.9321372361357556), ('she', 0.931748191682991), ('there', 0.931474501615315), ('body', 0.9314325201133546)]
------------------------------------------------------
OPT:
[('female', 0.8555820231335811), ('male', 0.8261287130890589), ('human', 0.8241803935686773), ('adult', 0.8069231736537754), ('humanitarian', 0.7793348043752667), ('humanity', 0.7541080142445051), ('youngster', 0.689500163722857), ('foreigner', 0.6710781843454334), ('individual', 0.6632535970659585), ('realistic', 0.6566244547032107), ('normal', 0.6552218875540489), ('professional', 0.653634253866648)]
------------------------------------------------------
T5:
[('human', 0.8946431198333841), ('adult', 0.8882116496784859), ('female', 

### D - Philosophy, math, miscellaneous
The rest of our old tests.

In [114]:
define(['logical', 'belief'])

logical, belief
GPT-3:
[('belief', 0.9610878447300748), ('logical', 0.961087840777404), ('logic', 0.9514226994050056), ('believe', 0.9365886738707129), ('rational', 0.9220866661017761), ('rationals', 0.9189949606958676), ('according', 0.9176109561431749), ('log', 0.9102034461425076), ('faith', 0.909647427667679), ('based', 0.9075127407145084), ('argument', 0.9051264935741397), ('mental', 0.9040936289660271)]
------------------------------------------------------
OPT:
[('logical', 0.8686298650874484), ('belief', 0.8638181611108668), ('logic', 0.8153329468645325), ('believe', 0.7535396929609106), ('logo', 0.7212118866197624), ('log', 0.7145808577793147), ('belong', 0.7132726393135485), ('rationals', 0.6762045771949032), ('beloved', 0.6710317977246052), ('creativity', 0.653287596117461), ('significance', 0.6526555944191521), ('allegation', 0.6507775657382628)]
------------------------------------------------------
T5:
[('belief', 0.8626637020096586), ('logical', 0.8226030300955474), ('bel

In [115]:
sim_test(['logical', 'belief'], 'argument')

logical, belief -> argument
GPT-3:
0.9051264935741397
------------------------------------------------------
OPT:
0.5709569182759853
------------------------------------------------------
T5:
0.7157578839089455
------------------------------------------------------


In [116]:
define(['logical', 'reason'])

logical, reason
GPT-3:
[('reason', 0.9693700977301596), ('logical', 0.96937009597118), ('reasoning', 0.9617079100204993), ('logic', 0.9615034170979726), ('rational', 0.9347915484417193), ('rationals', 0.9300742448573999), ('according', 0.9231377174701464), ('log', 0.9186502160929122), ('argument', 0.9134936283348167), ('explanation', 0.9093026940337412), ('basis', 0.9086521662118081), ('right', 0.9076618129329159)]
------------------------------------------------------
OPT:
[('reason', 0.8735740793682134), ('reasoning', 0.8668881814997871), ('logical', 0.800214984556957), ('logic', 0.7808850381009105), ('reasonably', 0.7564182313054368), ('rationals', 0.7224550493248428), ('logo', 0.7101779272717026), ('significance', 0.6650287045278758), ('realization', 0.6589889476302537), ('mechanism', 0.6563013547201834), ('problematic', 0.6554704593368375), ('momentum', 0.6516133405259462)]
------------------------------------------------------
T5:
[('reason', 0.8971659086068241), ('logical', 0.84

In [117]:
sim_test(['logical', 'reason'], 'argument')

logical, reason -> argument
GPT-3:
0.9134936283348167
------------------------------------------------------
OPT:
0.634841584397775
------------------------------------------------------
T5:
0.7437359967037392
------------------------------------------------------


In [118]:
define(['thinking', 'argument'])

thinking, argument
GPT-3:
[('thinking', 0.9645651540086648), ('argument', 0.9645651532053392), ('think', 0.9336006712802247), ('argue', 0.9331700185021929), ('thought', 0.9239617396670978), ('reasoning', 0.9232342940745594), ('discussion', 0.9200451773656071), ('disagreement', 0.9187194310013898), ('mind', 0.9167488810747582), ('mental', 0.9158747483930274), ('disagree', 0.9116392823398809), ('mentally', 0.9113014224738654)]
------------------------------------------------------
OPT:
[('thinking', 0.85832505231564), ('argument', 0.8299727189690524), ('reasoning', 0.7707575634181968), ('consideration', 0.7406627609148346), ('think', 0.7208224748408821), ('meaning', 0.7174668874963145), ('thoughtful', 0.7147189346759638), ('interpretation', 0.7023427254931353), ('meaningful', 0.6980110172162385), ('likelihood', 0.697478083863454), ('realization', 0.692979657506685), ('argue', 0.6895400103543279)]
------------------------------------------------------
T5:
[('argument', 0.9023236991792), (

In [119]:
sim_test(['thinking', 'argument'], 'philosophy')

thinking, argument -> philosophy
GPT-3:
0.8816975730706829
------------------------------------------------------
OPT:
0.44441454177721684
------------------------------------------------------
T5:
0.6487898994846598
------------------------------------------------------


In [120]:
sim_test(['thought', 'argument'], 'philosophy')

thought, argument -> philosophy
GPT-3:
0.8656715980930316
------------------------------------------------------
OPT:
0.44721387417031166
------------------------------------------------------
T5:
0.6879967328150877
------------------------------------------------------


In [121]:
define(['true', 'belief'])

true, belief
GPT-3:
[('belief', 0.9626506219300575), ('true', 0.9626506182450418), ('believe', 0.9464591415300987), ('truth', 0.9322470276916108), ('faith', 0.9198031186453355), ('according', 0.9184875540402915), ('false', 0.914689309761547), ('based', 0.9146424089166816), ('there', 0.9106905022760909), ('right', 0.9093406745360855), ('worth', 0.9039789881547333), ('trust', 0.9036203330362664)]
------------------------------------------------------
OPT:
[('true', 0.8776498173594941), ('belief', 0.7587352896818277), ('believe', 0.744462529925344), ('realization', 0.7129266084613131), ('realize', 0.6886078718061539), ('beloved', 0.6830389136615618), ('faithful', 0.6673484955798893), ('aware', 0.6554524470273477), ('realistic', 0.6537691718697896), ('passionate', 0.6401056404542197), ('meaningful', 0.630021357095164), ('realm', 0.6272616348571322)]
------------------------------------------------------
T5:
[('belief', 0.8829381467770511), ('true', 0.879826886563743), ('believe', 0.8405949

In [122]:
sim_test(['true', 'belief'], 'knowledge')

true, belief -> knowledge
GPT-3:
0.8694230482643159
------------------------------------------------------
OPT:
0.5352892469969255
------------------------------------------------------
T5:
0.7174079165293603
------------------------------------------------------


In [123]:
define(['spiritual', 'belief'])

spiritual, belief
GPT-3:
[('belief', 0.9634544849523248), ('spiritual', 0.9634544822623605), ('spirit', 0.9547667386868425), ('faith', 0.9429499552155304), ('believe', 0.9335023829296828), ('religion', 0.9333384000406882), ('religious', 0.9286691489151747), ('divine', 0.9096353393627675), ('mental', 0.9077698531531245), ('inner', 0.9041892137653265), ('mentally', 0.9039988431487755), ('revelation', 0.9030688863356887)]
------------------------------------------------------
OPT:
[('spiritual', 0.8703473787017171), ('belief', 0.8379840190121464), ('faithful', 0.7299873199326213), ('spirit', 0.7242791494479701), ('believe', 0.7088020768369886), ('faith', 0.7080550983912672), ('beloved', 0.6996959765130097), ('ritual', 0.6956778376924695), ('consciousness', 0.6865914842338622), ('passionate', 0.6771348619774339), ('creativity', 0.6631968811641865), ('assurance', 0.6558377123206482)]
------------------------------------------------------
T5:
[('spiritual', 0.8975988865980589), ('belief', 0.

In [124]:
sim_test(['spiritual', 'belief'], 'religion')

spiritual, belief -> religion
GPT-3:
0.9333384000406882
------------------------------------------------------
OPT:
0.644614827550498
------------------------------------------------------
T5:
0.6500590816624514
------------------------------------------------------


In [125]:
define(['right', 'belief'])

right, belief
GPT-3:
[('belief', 0.9646995410703604), ('right', 0.9646995368584058), ('believe', 0.9442840271355152), ('according', 0.938975679374828), ('there', 0.9356063454511181), ('you', 0.9333479716492321), ('based', 0.9305922920207241), ('faith', 0.9305431498599339), ('rid', 0.9287000047032449), ('mind', 0.9284612682969824), ('they', 0.9273620177454093), ('how', 0.9266004192125916)]
------------------------------------------------------
OPT:
[('right', 0.8866324460746231), ('rightly', 0.7999700050319551), ('belief', 0.7449513320983696), ('believe', 0.7139679815481608), ('left', 0.6725345722774626), ('correct', 0.6670752081179945), ('belong', 0.6456947179407113), ('beloved', 0.6433547853848174), ('relaxed', 0.6252687183662787), ('delighted', 0.6189343152294868), ('manipulation', 0.6184168850643155), ('passionate', 0.6171132962204988)]
------------------------------------------------------
T5:
[('right', 0.8568094380206241), ('belief', 0.8364688850886154), ('believe', 0.75930363078

In [126]:
sim_test(['right', 'belief'], 'moral')

right, belief -> moral
GPT-3:
0.8822403856676478
------------------------------------------------------
OPT:
0.4007069695355071
------------------------------------------------------
T5:
0.5845281452674516
------------------------------------------------------


In [127]:
sim_test(['right', 'belief'], 'ideal')

right, belief -> ideal
GPT-3:
0.878571300666416
------------------------------------------------------
OPT:
0.5667773795125282
------------------------------------------------------
T5:
0.615437920506338
------------------------------------------------------


In [128]:
define(['good', 'evil', 'belief'])

good, evil, belief
GPT-3:
[('evil', 0.9479486514294942), ('there', 0.9440605116866795), ('good', 0.9417385145930361), ('well', 0.9408187380314945), ('you', 0.9392227833956983), ('belief', 0.9384015077116685), ('mind', 0.9382525528956378), ('how', 0.9365043030035465), ('list', 0.9351974044461806), ('what', 0.934794975067752), ('rid', 0.9346818520990274), ('based', 0.9345843778015237)]
------------------------------------------------------
OPT:
[('good', 0.837036770539628), ('goodness', 0.8301249899664292), ('goods', 0.8134598560278434), ('evil', 0.7871775329910292), ('goodbye', 0.7719366754469573), ('badge', 0.7424197334309952), ('bad', 0.7415730985138389), ('solidarity', 0.7075685884228109), ('great', 0.7071876476704924), ('bad-tempered', 0.7047205597257535), ('powerful', 0.6961565311966451), ('outstanding', 0.6950207456190802)]
------------------------------------------------------
T5:
[('evil', 0.8844895165687652), ('good', 0.8374045851531069), ('bad', 0.8181901784041558), ('terrible

In [129]:
sim_test(['good', 'evil', 'belief'], 'moral')

good, evil, belief -> moral
GPT-3:
0.9032934647971853
------------------------------------------------------
OPT:
0.4743267189977657
------------------------------------------------------
T5:
0.7144806348814423
------------------------------------------------------


## Define Helpers (for magnitude)

In [130]:
# this function doesn't allow model specification, as GPT-3 vectors are normalized.
def sum_complexity(word, vec=False, model='opt'):
    """
    Args:
        word: iterable, string, or vector   -> embedding to be summed.
        vec: bool                           -> True if word is already vector.
        model: str                          -> 'opt' or 't5'. No gpt weights.
    Returns:
        Float score of the sum of the absolute value of each datapoint in the embedding.
    """
    phrase = word
    if not vec:
        if isinstance(word, str):
            phrase = positive([word], model=model)
        else:
            phrase = positive(word, model=model)
    
    modified = [abs(float(x)) for x in phrase[0]]
    return sum(modified)
    

In [131]:
def above_zero_complexity(word, vec=False, threshold=0.3, model='opt'):
    """
    Args:
        word: iterable, string, or vector   -> embedding to be evaluated
        vec: bool                           -> True if word is already vector.
        threshold: float                    -> Absolute value cutoff point
        model: str                          -> 'opt' or 't5'. No gpt weights.
    Returns:
        Float score of the amount of 'signifigant' features according to the threshold,
        divided by the number of total dimensions.
    """
    phrase = word
    if not vec:
        if isinstance(word, str):
            phrase = positive([word], model=model)
        else:
            phrase = positive(word, model=model)

    modified = [abs(float(x)) for x in phrase[0] if abs(float(x)) > threshold]
    return len(modified) / phrase.shape[1]

In [132]:
def compare_complexity(words, threshold=0.3):
    """
    Args:
        words: iterable     -> items to be converted to tokens and compared.
        threshold: fload    -> threshold for signifigant features with a_z_c()
    Returns:
        None. Formatted printing results for each item in words.
    """
    for w in words:
        print(w)
        print("OPT-1.3b:")
        print(f"Sum Complexity: {sum_complexity(w)}")
        print(f"Above Zero Complexity t-[{threshold}]: {above_zero_complexity(w, threshold=threshold)}")
        print('--------------------------------------------------')
        print("T5:")
        print(f"Sum Complexity: {sum_complexity(w, model='t5')}")
        print(f"Above Zero Complexity t-[{threshold}]: {above_zero_complexity(w, threshold=threshold, model='t5')}")
        print('--------------------------------------------------')
        print()

## Conduct Magnitude Tests
This is a shorter section, and all of our tests are on OPT-1.3b.

In [133]:
compare_complexity(['man', 'bachelor', 'husband', 'father'])

man
OPT-1.3b:
Sum Complexity: 3869.585996808201
Above Zero Complexity t-[0.3]: 0.88427734375
--------------------------------------------------
T5:
Sum Complexity: 111.46183970193012
Above Zero Complexity t-[0.3]: 0.03125
--------------------------------------------------

bachelor
OPT-1.3b:
Sum Complexity: 2698.8531398967007
Above Zero Complexity t-[0.3]: 0.83837890625
--------------------------------------------------
T5:
Sum Complexity: 116.65366553346695
Above Zero Complexity t-[0.3]: 0.0419921875
--------------------------------------------------

husband
OPT-1.3b:
Sum Complexity: 3166.151641501363
Above Zero Complexity t-[0.3]: 0.86865234375
--------------------------------------------------
T5:
Sum Complexity: 114.48292873662996
Above Zero Complexity t-[0.3]: 0.0380859375
--------------------------------------------------

father
OPT-1.3b:
Sum Complexity: 3597.1001238524773
Above Zero Complexity t-[0.3]: 0.88232421875
--------------------------------------------------
T5:
Sum Co

In [134]:
compare_complexity(['woman', 'wife', 'mother'])

woman
OPT-1.3b:
Sum Complexity: 3872.539441489794
Above Zero Complexity t-[0.3]: 0.89453125
--------------------------------------------------
T5:
Sum Complexity: 112.19001309328294
Above Zero Complexity t-[0.3]: 0.04296875
--------------------------------------------------

wife
OPT-1.3b:
Sum Complexity: 3150.2644422225003
Above Zero Complexity t-[0.3]: 0.86572265625
--------------------------------------------------
T5:
Sum Complexity: 120.24246082537995
Above Zero Complexity t-[0.3]: 0.0517578125
--------------------------------------------------

mother
OPT-1.3b:
Sum Complexity: 3577.550980384024
Above Zero Complexity t-[0.3]: 0.88623046875
--------------------------------------------------
T5:
Sum Complexity: 120.90871189110396
Above Zero Complexity t-[0.3]: 0.056640625
--------------------------------------------------



In [135]:
compare_complexity(['cat', 'kitten'])

cat
OPT-1.3b:
Sum Complexity: 3582.2652872879603
Above Zero Complexity t-[0.3]: 0.8984375
--------------------------------------------------
T5:
Sum Complexity: 108.42372198904003
Above Zero Complexity t-[0.3]: 0.03515625
--------------------------------------------------

kitten
OPT-1.3b:
Sum Complexity: 2721.951363982246
Above Zero Complexity t-[0.3]: 0.8505859375
--------------------------------------------------
T5:
Sum Complexity: 89.01718946458
Above Zero Complexity t-[0.3]: 0.0048828125
--------------------------------------------------



In [136]:
compare_complexity(['dog', 'puppy'])

dog
OPT-1.3b:
Sum Complexity: 3529.7578563156044
Above Zero Complexity t-[0.3]: 0.87548828125
--------------------------------------------------
T5:
Sum Complexity: 108.30607459486987
Above Zero Complexity t-[0.3]: 0.029296875
--------------------------------------------------

puppy
OPT-1.3b:
Sum Complexity: 2501.963246831801
Above Zero Complexity t-[0.3]: 0.8310546875
--------------------------------------------------
T5:
Sum Complexity: 90.12560882247998
Above Zero Complexity t-[0.3]: 0.00390625
--------------------------------------------------



In [137]:
compare_complexity(['duck', 'duckling'])

duck
OPT-1.3b:
Sum Complexity: 2385.5245381011996
Above Zero Complexity t-[0.3]: 0.83203125
--------------------------------------------------
T5:
Sum Complexity: 102.58098223756015
Above Zero Complexity t-[0.3]: 0.021484375
--------------------------------------------------

duckling
OPT-1.3b:
Sum Complexity: 2413.2867072275394
Above Zero Complexity t-[0.3]: 0.822265625
--------------------------------------------------
T5:
Sum Complexity: 80.48213390689506
Above Zero Complexity t-[0.3]: 0.0126953125
--------------------------------------------------



In [138]:
compare_complexity(['set', 'powerset'])

set
OPT-1.3b:
Sum Complexity: 3559.9400755744996
Above Zero Complexity t-[0.3]: 0.8818359375
--------------------------------------------------
T5:
Sum Complexity: 103.08317470063999
Above Zero Complexity t-[0.3]: 0.0205078125
--------------------------------------------------

powerset
OPT-1.3b:
Sum Complexity: 2987.547082962999
Above Zero Complexity t-[0.3]: 0.861328125
--------------------------------------------------
T5:
Sum Complexity: 113.69557875905991
Above Zero Complexity t-[0.3]: 0.0390625
--------------------------------------------------



In [139]:
compare_complexity(['matrix', 'determinant'])

matrix
OPT-1.3b:
Sum Complexity: 2349.8595956072013
Above Zero Complexity t-[0.3]: 0.83349609375
--------------------------------------------------
T5:
Sum Complexity: 115.38173912045986
Above Zero Complexity t-[0.3]: 0.0478515625
--------------------------------------------------

determinant
OPT-1.3b:
Sum Complexity: 2404.382848897932
Above Zero Complexity t-[0.3]: 0.814453125
--------------------------------------------------
T5:
Sum Complexity: 96.57637868054012
Above Zero Complexity t-[0.3]: 0.017578125
--------------------------------------------------



## Phrase comparison
In the past, we have not had access to models that make a phrase, so we use positives. I will explore some of those combinations below.
Phrases of interest:
* "white house"
* "baby duck"
* "logical belief"

and so on...

In [140]:
define(['logical belief']) # this usage should work.

logical belief
GPT-3:
[('logical', 0.9368985262715313), ('belief', 0.9201455360856643), ('logic', 0.9101838649160616), ('rationals', 0.8914305041137053), ('believe', 0.8895525995505191), ('rational', 0.8881465730039954), ('philosophical', 0.8771653968799995), ('reasoning', 0.8712889059448021), ('religion', 0.8698503345332375), ('conviction', 0.8680595632427539), ('argument', 0.8660604165608106), ('ideological', 0.8634520187388373)]
------------------------------------------------------
OPT:
[('logical', 0.9087944009447604), ('logic', 0.825824372696814), ('logo', 0.7385080961175052), ('rationals', 0.7075011537887733), ('log', 0.6672627357815122), ('consciousness', 0.6666232841396709), ('assertion', 0.6629146259128236), ('complexity', 0.659734467248192), ('rational', 0.6516858549956231), ('ideology', 0.648526383521467), ('realization', 0.6413178876611093), ('validity', 0.6354558918953294)]
------------------------------------------------------
T5:
[('logical', 0.8212626895989883), ('defi

In [141]:
sim_test(["white", "house"], 'white house') # this is also valid!

white, house -> white house
GPT-3:
0.9318936864551356
------------------------------------------------------
OPT:
0.8336804090210822
------------------------------------------------------
T5:
0.6817424802191578
------------------------------------------------------


In [142]:
sim_test(['white house'], 'Joe Biden') # we can even do unique, new tests with our wrappers!

white house -> Joe Biden
GPT-3:
0.8575577085369428
------------------------------------------------------
OPT:
0.3369971
------------------------------------------------------
T5:
0.4775062
------------------------------------------------------


In [143]:
define(['baby duck'])

baby duck
GPT-3:
[('duckling', 0.9390343440641757), ('baby', 0.9264423570299711), ('duck', 0.909063230723647), ('infant', 0.8880650616731081), ('puppy', 0.8766010076739243), ('child', 0.872585765492617), ('pregnancy', 0.8684074578100834), ('nursery', 0.8677849698698282), ('offspring', 0.867696413163072), ('children', 0.8671002494515161), ('birth', 0.8638819733659289), ('chicken', 0.8615226407135073)]
------------------------------------------------------
OPT:
[('baby', 0.8877308969945594), ('duckling', 0.7398302440354387), ('boyfriend', 0.6958211796219), ('daughter', 0.6850545648357198), ('youngster', 0.6530324481002916), ('girl', 0.6430866280437579), ('sheep', 0.6389098997387503), ('firefighter', 0.6386129605797467), ('kidney', 0.635517633010759), ('granddaughter', 0.6354172100385012), ('grandson', 0.6352220595348049), ('chairwoman', 0.6334669978202907)]
------------------------------------------------------
T5:
[('duck', 0.5120337666900932), ('dancer', 0.48856933384392875), ('boar', 

In [144]:
sim_test(['baby', 'duck'], 'baby duck')

baby, duck -> baby duck
GPT-3:
0.9529651714836083
------------------------------------------------------
OPT:
0.885532781089644
------------------------------------------------------
T5:
0.5097756929156112
------------------------------------------------------


In [145]:
sim_test(['baby duck'], 'duckling')

baby duck -> duckling
GPT-3:
0.9390343440641757
------------------------------------------------------
OPT:
0.7398302440354387
------------------------------------------------------
T5:
0.44453311461490463
------------------------------------------------------


As some brief discussion before submit, some of these scores are very surprising. That final sim_test has me scratching my head a bit, why did OPT score it at 0.19?

There are different sizes of OPT to be tried, but I like the 2048-D vectors from OPT-1.3B, and it didn't use too much cache space. We can (and should) test this on larger versions (Paramenter size can be changed- ['125m', '350m', '1.3b', '2.7b', '6.7b', '13b', '30b']). For reference, 350M has 512-D vectors. If there is an error in vector generation I will fix it, I hope this is easy to use and read.

Notes:
* Average Similarity? How does this look, how can we calculate
* Use larger OPT models? Cache them?
* Phrase tests :)
    * sentences?

Early development and language development - which words and concepts come first?

In [146]:
def phrase_comparison(phrases):
    """
    Args:
        phrases: iterable   -> iterable of novel phrases to compare.
    Returns:
        None. Formats and facilitates phrase comparison of an iterable of novel phrases.
    """
    enc = [(positive([str(x)], 'gpt'), positive([str(x)], 'opt'), positive([str(x)], 't5')) for x in phrases]
    print(*phrases, sep=', ')
    for i in range(len(phrases)):
        for j in range(i+1, len(phrases)):
            print(f"{phrases[i]} -> {phrases[j]}")
            print(f"GPT-3: {cosine_similarity(enc[i][0], enc[j][0])[0][0]}")
            print(f"OPT-1.3B: {cosine_similarity(enc[i][1], enc[j][1])[0][0]}")
            print(f"T5: {cosine_similarity(enc[i][2], enc[j][2])[0][0]}")
            print('------------------------------------------------------')
    print("\nPhrase Definitions:")
    for p in phrases:
        define([p])

### A - spouse phrases

In [147]:
phrase_comparison(['unmarried man', 'a bachelor is an unmarried man'])

unmarried man, a bachelor is an unmarried man
unmarried man -> a bachelor is an unmarried man
GPT-3: 0.9268371372784423
OPT-1.3B: 0.5956459641456604
T5: 0.595443606376648
------------------------------------------------------

Phrase Definitions:
unmarried man
GPT-3:
[('unmarried', 0.958565179630738), ('husband', 0.9041219456247627), ('unemployed', 0.9003319097823591), ('married', 0.8971223416424852), ('marriage', 0.8904060822674948), ('divorced', 0.8844611500405011), ('single', 0.8780193993602652), ('man', 0.8745131822236145), ('unemployment', 0.8744712571127441), ('male', 0.8742086226015324), ('spouse', 0.8727524284215669), ('manifest', 0.8723020505820172)]
------------------------------------------------------
OPT:
[('unmarried', 0.903176760345992), ('unemployed', 0.824555108771978), ('unsuccessful', 0.7371520912662628), ('unconscious', 0.7322927791683297), ('unhappy', 0.7198057464858689), ('unimportant', 0.7096539906384891), ('unacceptable', 0.703186976952224), ('university', 0.701

In [148]:
phrase_comparison(['adult female horse', 'Look, a mare!'])

adult female horse, Look, a mare!
adult female horse -> Look, a mare!
GPT-3: 0.8950975294257666
OPT-1.3B: 0.4912247657775879
T5: 0.44851261377334595
------------------------------------------------------

Phrase Definitions:
adult female horse
GPT-3:
[('horse', 0.9042642876415965), ('female', 0.8568780313596931), ('carriage', 0.8501774726769284), ('riding', 0.8414691039193982), ('sheep', 0.8371935048730068), ('chairwoman', 0.8368366526485758), ('woman', 0.8360628241265586), ('rider', 0.8330991681011948), ('adult', 0.8271051053104326), ('animal', 0.8221076338118947), ('granddaughter', 0.8220295043522654), ('hairdresser', 0.8184311273395052)]
------------------------------------------------------
OPT:
[('adult', 0.8141187165612553), ('female', 0.7818205934593554), ('male', 0.7615717287698658), ('youngster', 0.6994360047281498), ('daughter', 0.663698585894489), ('horse', 0.6575905023972899), ('boyfriend', 0.6523049854743344), ('foreigner', 0.6475792845417278), ('woman', 0.6381339774049464

In [149]:
phrase_comparison(['What do you call an unmarried man?', 'What do you call a married man?'])

What do you call an unmarried man?, What do you call a married man?
What do you call an unmarried man? -> What do you call a married man?
GPT-3: 0.9269945921124785
OPT-1.3B: 0.9753906726837158
T5: 0.8821046948432922
------------------------------------------------------

Phrase Definitions:
What do you call an unmarried man?
GPT-3:
[('unmarried', 0.8610593561976344), ('unemployed', 0.8254371850409273), ('alone', 0.8154799776370714), ('single', 0.8103635924527604), ('anonymous', 0.8094950939498504), ('male', 0.8008065570977274), ('unemployment', 0.7977282492695694), ('husband', 0.7951773283497702), ('lonely', 0.7939977675552856), ('bachelor', 0.7919560788765844), ('outsider', 0.7911840801125203), ('gentleman', 0.7909655688610424)]
------------------------------------------------------
OPT:
[('whereas', 0.740855450924762), ('including', 0.7236770261867851), ('exceptional', 0.716305283835764), ('because', 0.7058284606346992), ('whereby', 0.7055510728195485), ('excluding', 0.70411634159165

In [150]:
phrase_comparison(['My dog had babies!', 'Do you want to play with a puppy?'])

My dog had babies!, Do you want to play with a puppy?
My dog had babies! -> Do you want to play with a puppy?
GPT-3: 0.8485986163569736
OPT-1.3B: 0.6416595578193665
T5: 0.5311905145645142
------------------------------------------------------

Phrase Definitions:
My dog had babies!
GPT-3:
[('puppy', 0.8656858136738301), ('dog', 0.8615180869451775), ('pregnancy', 0.8482552549382141), ('adoption', 0.8474235711829837), ('pregnant', 0.840981554553738), ('baby', 0.8403259061265272), ('animal', 0.8329189687747917), ('feeding', 0.8305430673063319), ('kitten', 0.8293582473223671), ('birth', 0.8277779904555372), ('offspring', 0.8269478986507568), ('conception', 0.8247380824939364)]
------------------------------------------------------
OPT:
[('myself', 0.6384426541557902), ('yesterday', 0.5888876447803151), ('i.e.', 0.5816656088244625), ('and', 0.5781613786207983), ('afterwards', 0.5770340917274326), ('yeah', 0.5759290788381608), ('either', 0.5739918805674212), ('including', 0.570936631228183),

In [151]:
phrase_comparison([
    'Knowledge is a true belief.', 
    'An argument is a logical belief.',
    'A spiritual belief is religion.'
])

Knowledge is a true belief., An argument is a logical belief., A spiritual belief is religion.
Knowledge is a true belief. -> An argument is a logical belief.
GPT-3: 0.858528136565669
OPT-1.3B: 0.5583148002624512
T5: 0.6611995697021484
------------------------------------------------------
Knowledge is a true belief. -> A spiritual belief is religion.
GPT-3: 0.8706939048506916
OPT-1.3B: 0.6498347520828247
T5: 0.6728665828704834
------------------------------------------------------
An argument is a logical belief. -> A spiritual belief is religion.
GPT-3: 0.8733140441620892
OPT-1.3B: 0.6986019611358643
T5: 0.7070983648300171
------------------------------------------------------

Phrase Definitions:
Knowledge is a true belief.
GPT-3:
[('knowledge', 0.8724948516118745), ('belief', 0.8673685567468519), ('believe', 0.842950572721898), ('conviction', 0.8385328002010175), ('true', 0.8384210793621812), ('wisdom', 0.8350698519598289), ('truth', 0.831368963641485), ('credible', 0.8290127816985

## Average Similarity
What is the average similarity between words in our dictionary for each model?

In [152]:
def make_avg_tbl(model='opt', extra_vocab=[]):
    """
    Args:
        model: str  -> 'gpt', 'opt', or 't5'
        extra_vocab: iterable of extra strings to be processed like vocab
    Returns:
        Table that contains the average cosine similarity of each word in our vocabulary
        to each other word, possibly including user supplied extra vocab.
    """
    print(f"Making average table for model: {model}")
    tbl_vocab = [str(x) for x in vocab + list(extra_vocab)]
    tbl_sz = len(tbl_vocab)
    table = {k: 0.0 for k in tbl_vocab}
    for term_a in tbl_vocab:
        a = positive([term_a], model)
        for term_b in tbl_vocab:
            if term_a != term_b:
                b = positive([term_b], model)
                table[term_a] += cosine_similarity(a, b)[0][0]
        table[term_a] /= tbl_sz
    print('done')
    return table                

In [153]:
def calc_avg_sim():
    # Does this need a header? Wrapper.
    print("What is our average connection strength?")
    gpt_tbl = make_avg_tbl(model='gpt')
    t5_tbl = make_avg_tbl(model='t5')
    opt_tbl = make_avg_tbl()
    sz = len(opt_tbl)
    print("GPT-3:")
    print(sum(gpt_tbl.values()) / sz)
    print('------------------------------------------------------')
    print('OPT-1.3B:')
    print(sum(opt_tbl.values()) / sz)
    print('------------------------------------------------------')
    print('T5:')
    print(sum(t5_tbl.values()) / sz)
    print('------------------------------------------------------')

In [154]:
calc_avg_sim()
# takes ~500m

What is our average connection strength?
Making average table for model: gpt
done
Making average table for model: t5
done
Making average table for model: opt
done
GPT-3:
0.8170242997763668
------------------------------------------------------
OPT-1.3B:
0.38555927651244076
------------------------------------------------------
T5:
0.4803895017774575
------------------------------------------------------
